# Event-Based Portfolio Backtesting System

**Strategy:** Trend Following with Event-Driven Architecture  
**Universe:** S&P 500 (SPY) Constituent Stocks  
**Period:** January 2010 - December 2019  
**Initial Capital:** $500,000 USD  
**Transaction Cost:** 0%  
**Leverage:** 0x  

---

## 1. Setup and Imports

In [144]:
# Install required packages
%pip install pandas numpy yfinance matplotlib seaborn ta scipy tqdm -q ipywidgets

import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import seaborn as sns
# from ta.momentum import RSIIndicator  <- REMOVED
from ta.trend import EMAIndicator #, ADXIndicator <- REMOVED
from ta.volatility import AverageTrueRange, BollingerBands
from typing import Dict, List, Optional, Tuple
from dataclasses import dataclass, field
from enum import Enum
from datetime import datetime, timedelta
import warnings
from tqdm.notebook import tqdm
import json

warnings.filterwarnings('ignore')
sns.set_style('darkgrid')

%matplotlib inline
plt.rcParams['figure.figsize'] = (15, 6)

print("✅ All imports successful!")


@dataclass
class StrategyConfig:
    """Strategy configuration parameters"""
    # Technical indicator parameters
    ema_fast: int = 20
    ema_slow: int = 50
    # rsi_period: int = 14  <- REMOVED
    # adx_period: int = 14  <- REMOVED
    atr_period: int = 14  # Kept for stop-loss calculation
    
    # Bollinger Bands parameters
    bb_period: int = 20
    bb_std: float = 1.5
    
    # adx_threshold: float = 25.0       <- REMOVED
    
    # Mean Reversion parameters
    # mr_profit_target: float = 0.04    <- REMOVED
    # mr_stop_loss: float = 0.01      <- REMOVED
    
    # Regime parameters
    vix_regime_threshold: float = 25.0  # <- SIMPLIFIED
    vix_smoothing_period: int = 10
    
    # Risk management
    stop_loss_pct: float = 0.02  # 2% (or 2x ATR if using ATR-based stops)
    trailing_stop_pct: float = 0.015  # 1.5% (or 1.5x ATR)
    max_portfolio_drawdown: float = 0.15  # 15%
    risk_per_trade: float = 0.04 # 2%
    min_position_size: float = 0.05  # 10%
    max_position_size: float = 0.1 # 20% per stock (for portfolio)
    
    # Portfolio parameters
    initial_capital: float = 500000.0  # $500K
    transaction_cost: float = 0.0  # 0%
    leverage: float = 0.0  # No leverage
    max_positions: int = 10  # Max concurrent positions
    
    # Data parameters
    train_start: str = "2010-01-01"
    train_end: str = "2019-12-31"


class MarketRegime(Enum):
    """Market volatility regimes"""
    # SIMPLIFIED to two strategy-based regimes
    MEAN_REVERSION = "mean_reversion"  # VIX <= 30
    TREND_FOLLOWING = "trend_following"   # VIX > 30


class EventType(Enum):
    """Types of market events"""
    # Regime events
    REGIME_CHANGE = "regime_change"
    
    # Trend following events
    GOLDEN_CROSS = "golden_cross"
    DEATH_CROSS = "death_cross"
    # ADX_STRONG_TREND = "adx_strong_trend" <- REMOVED
    # ADX_WEAK_TREND = "adx_weak_trend"   <- REMOVED
    
    # Mean reversion events
    BB_UPPER_CROSS = "bb_upper_cross"      
    BB_LOWER_CROSS = "bb_lower_cross"      
    BB_MIDDLE_CROSS_UP = "bb_middle_cross_up"    
    BB_MIDDLE_CROSS_DOWN = "bb_middle_cross_down"  
    # BB_SQUEEZE = "bb_squeeze"              <- REMOVED
    # BB_EXPANSION = "bb_expansion"          <- REMOVED
    
    # RSI events (ALL REMOVED)
    # RSI_OVERBOUGHT = "rsi_overbought"
    # RSI_OVERSOLD = "rsi_oversold"
    # RSI_EXIT_OVERBOUGHT = "rsi_exit_overbought"
    # RSI_EXIT_OVERSOLD = "rsi_exit_oversold"
    
    # Exit events
    STOP_LOSS_HIT = "stop_loss_hit"
    TRAILING_STOP_HIT = "trailing_stop_hit"
    MAX_DRAWDOWN_HIT = "max_drawdown_hit"
    # PROFIT_TARGET_HIT = "profit_target_hit" <- REMOVED
    ORDER_FILLED = "order_filled"


@dataclass
class Event:
    """Represents a market event"""
    timestamp: datetime
    event_type: EventType
    symbol: str
    data: Dict
    priority: int = 0
    
    def __repr__(self):
        return f"Event({self.timestamp.date()}, {self.symbol}, {self.event_type.value}, priority={self.priority})"


@dataclass
class Position:
    """Represents an open position"""
    symbol: str
    side: str  # 'LONG' or 'SHORT'
    entry_price: float
    entry_time: datetime
    shares: float
    stop_loss: float
    trailing_stop: float
    highest_price: float = 0.0
    lowest_price: float = float('inf')
    regime: MarketRegime = MarketRegime.MEAN_REVERSION # Default
    
    @property
    def value(self) -> float:
        """Current position value"""
        return self.shares * self.entry_price


@dataclass
class Trade:
    """Completed trade record"""
    symbol: str
    side: str
    entry_time: datetime
    exit_time: datetime
    entry_price: float
    exit_price: float
    shares: float
    pnl: float
    pnl_pct: float
    reason: str
    regime: MarketRegime


print("✅ Configuration and data classes defined!")


def get_spy_stocks(top_n: int = 50) -> List[str]:
    """
    Get SPY constituent stocks
    For this example, we'll use the top 50 most liquid S&P 500 stocks
    """
    # Top 50 S&P 500 stocks by market cap (as of 2010-2019 period)
    spy_stocks = [
        'AAPL', 'MSFT', 'AMZN', 'GOOGL', 'BRK.B', 'JNJ', 'JPM', 'V', 'PG', 'XOM',
        'BAC', 'MA', 'T', 'DIS', 'HD', 'INTC', 'VZ', 'PFE', 'MRK', 'CSCO',
        'KO', 'PEP', 'WFC', 'CMCSA', 'C', 'CVX', 'NFLX', 'ABT', 'ORCL', 'COST',
        'BA', 'MCD', 'IBM', 'UNH', 'PM', 'GE', 'HON', 'QCOM', 'AMGN', 'MDT',
        'UPS', 'BMY', 'SBUX', 'TXN', 'LOW', 'MMM', 'LMT', 'CAT', 'NKE', 'GS'
    ]
    return ['NVDA','AAPL', 'MSFT', 'AMZN', 'META', 'AVGO', 'GOOGL', 'TSLA', 'GOOG', 'BRK-B']


def load_vix_data(config: StrategyConfig) -> pd.DataFrame:
    """
    Load and process VIX data for regime detection
    Returns DataFrame with VIX data and regime indicators
    """
    # Download VIX data
    vix = yf.download("^VIX", 
                      start=config.train_start,
                      end=config.train_end,
                      progress=False)
    
    vix.index = vix.index.tz_localize(None)
    
    if not vix.index.is_unique:
        print("⚠️ Warning: VIX data contains duplicate index dates. Keeping first entry for each date.")
        vix = vix[~vix.index.duplicated(keep='first')]
        
    # Calculate smoothed VIX (simple moving average)
    vix['VIX_MA'] = vix['Close'].rolling(window=config.vix_smoothing_period).mean()
    
    # --- START OF SIMPLIFICATION ---
    # Define market regime based on the single threshold
    vix['Regime'] = np.where(
        vix['VIX_MA'] > config.vix_regime_threshold, 
        MarketRegime.TREND_FOLLOWING.value, 
        MarketRegime.MEAN_REVERSION.value
    )
    # --- END OF SIMPLIFICATION ---
    
    # Drop NaN values from moving average calculation
    vix = vix.dropna()
    
    return vix

# Get stock universe
SPY_STOCKS = get_spy_stocks(50)
print(f"✅ Using {len(SPY_STOCKS)} SPY stocks:")
print(f"   {', '.join(SPY_STOCKS[:10])}...")


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ All imports successful!
✅ Configuration and data classes defined!
✅ Using 10 SPY stocks:
   NVDA, AAPL, MSFT, AMZN, META, AVGO, GOOGL, TSLA, GOOG, BRK-B...


## 4. Technical Indicators

In [145]:
def calculate_indicators(df: pd.DataFrame, config: StrategyConfig) -> pd.DataFrame:
    """Calculate technical indicators for a stock"""
    data = pd.DataFrame(index=df.index)
    data['Close'] = df['Close']
    data['High'] = df['High']
    data['Low'] = df['Low']
    data['Volume'] = df['Volume']
    
    # EMAs
    data['EMA_Fast'] = EMAIndicator(df['Close'], window=config.ema_fast, fillna=True).ema_indicator()
    data['EMA_Slow'] = EMAIndicator(df['Close'], window=config.ema_slow, fillna=True).ema_indicator()
    data['SMA_200'] = data['Close'].rolling(window=200).mean()
    
    # RSI <- REMOVED
    # data['RSI'] = RSIIndicator(df['Close'], window=config.rsi_period, fillna=True).rsi()
    
    # ADX <- REMOVED
    # adx = ADXIndicator(df['High'], df['Low'], df['Close'], window=config.adx_period, fillna=True)
    # data['ADX'] = adx.adx()
    
    # ATR
    atr = AverageTrueRange(df['High'], df['Low'], df['Close'], window=config.atr_period, fillna=True)
    data['ATR'] = atr.average_true_range()
    data['ATR_Pct'] = data['ATR'] / data['Close']  # Kept for stop-loss
    
    # Bollinger Bands
    bb = BollingerBands(df['Close'], window=config.bb_period, window_dev=config.bb_std, fillna=True)
    data['BB_Upper'] = bb.bollinger_hband()
    data['BB_Lower'] = bb.bollinger_lband()
    data['BB_Middle'] = bb.bollinger_mavg()
    # data['BB_Width'] = (data['BB_Upper'] - data['BB_Lower']) / data['BB_Middle'] <- REMOVED
    # data['BB_Pct_B'] = (df['Close'] - data['BB_Lower']) / (data['BB_Upper'] - data['BB_Lower']) <- REMOVED
    
    # Drop initial NaN rows
    # SIMPLIFIED max_window calculation
    max_window = max(config.ema_slow, config.bb_period, config.atr_period)
    data = data.iloc[max_window:]
    
    return data

print("✅ SIMPLIFIED Technical indicators function defined!")

✅ SIMPLIFIED Technical indicators function defined!


## 5. Event Detector

In [146]:
class EventDetector:
    """
    Detects market events from price data.
    
    This SIMPLIFIED version only detects the core events
    for our two strategies: EMA crosses and BB crosses.
    """
    
    def __init__(self, config: StrategyConfig):
        self.config = config
        self.previous_state = {}
        

    def detect_mean_reversion_events(self, symbol: str, current_data: pd.Series, prev_data: pd.Series,
                                   timestamp: datetime) -> List[Event]:
        """Detect trend following events"""
        events = []
        
        # CROSSOVER EVENTS
        if (current_data['EMA_Fast'] > current_data['EMA_Slow'] and 
            prev_data['EMA_Fast'] <= prev_data['EMA_Slow']):
            events.append(Event(
                timestamp=timestamp,
                event_type=EventType.GOLDEN_CROSS,
                symbol=symbol,
                data={'price': current_data['Close']}, # <- SIMPLIFIED (removed rsi, adx)
                priority=10
            ))
        
        if (current_data['EMA_Fast'] < current_data['EMA_Slow'] and 
            prev_data['EMA_Fast'] >= prev_data['EMA_Slow']):
            events.append(Event(
                timestamp=timestamp,
                event_type=EventType.DEATH_CROSS,
                symbol=symbol,
                data={'price': current_data['Close']}, # <- SIMPLIFIED (removed rsi, adx)
                priority=10
            ))
        
        # ADX EVENTS <- REMOVED
        
        return events
    
    def detect_trend_events(self, symbol: str, current_data: pd.Series, prev_data: pd.Series,
                          timestamp: datetime) -> List[Event]:
        """Detect mean reversion events"""
        events = []
        
        # Bollinger Band crosses
        if (current_data['Close'] > current_data['BB_Upper'] and 
            prev_data['Close'] <= prev_data['BB_Upper']):
            events.append(Event(
                timestamp=timestamp,
                event_type=EventType.BB_UPPER_CROSS,
                symbol=symbol,
                data={'price': current_data['Close'], 'bb_upper': current_data['BB_Upper']}, # <- SIMPLIFIED (removed bb_width)
                priority=8
            ))
            
        if (current_data['Close'] < current_data['BB_Lower'] and 
            prev_data['Close'] >= prev_data['BB_Lower']):
            events.append(Event(
                timestamp=timestamp,
                event_type=EventType.BB_LOWER_CROSS,
                symbol=symbol,
                data={'price': current_data['Close'], 'bb_lower': current_data['BB_Lower']}, # <- SIMPLIFIED (removed bb_width)
                priority=8
            ))
        
        # Middle band crosses (potential exits)
        if (current_data['Close'] > current_data['BB_Middle'] and 
            prev_data['Close'] <= prev_data['BB_Middle']):
            events.append(Event(
                timestamp=timestamp,
                event_type=EventType.BB_MIDDLE_CROSS_UP,
                symbol=symbol,
                data={'price': current_data['Close'], 'bb_middle': current_data['BB_Middle']},
                priority=7
            ))
            
        if (current_data['Close'] < current_data['BB_Middle'] and 
            prev_data['Close'] >= prev_data['BB_Middle']):
            events.append(Event(
                timestamp=timestamp,
                event_type=EventType.BB_MIDDLE_CROSS_DOWN,
                symbol=symbol,
                data={'price': current_data['Close'], 'bb_middle': current_data['BB_Middle']},
                priority=7
            ))
        
        # Volatility events <- REMOVED
        
        return events
    
    def detect_regime_change(self, symbol: str, current_regime: MarketRegime, 
                           prev_regime: MarketRegime, timestamp: datetime) -> Optional[Event]:
        """Detect changes in market regime"""
        if current_regime != prev_regime:
            return Event(
                timestamp=timestamp,
                event_type=EventType.REGIME_CHANGE,
                symbol=symbol,
                data={'new_regime': current_regime.value, 'old_regime': prev_regime.value},
                priority=10  
            )
        return None
    
    def detect_events(self, symbol: str, current_data: pd.Series, prev_data: Optional[pd.Series],
                     current_regime: MarketRegime, prev_regime: MarketRegime) -> List[Event]:
        """
        Detect all events for a symbol at current timestamp
        """
        events = []
        
        if prev_data is None:
            return events
        
        timestamp = current_data.name
        
        # Check for regime change first
        regime_event = self.detect_regime_change(symbol, current_regime, prev_regime, timestamp)
        if regime_event:
            events.append(regime_event)
        
        # 1. Detect trend events
        events.extend(self.detect_trend_events(symbol, current_data, prev_data, timestamp))
        
        # 2. Detect mean reversion events
        events.extend(self.detect_mean_reversion_events(symbol, current_data, prev_data, timestamp))
        
        # 3. Detect RSI events <- REMOVED
        
        return events

print("✅ SIMPLIFIED Event detector class defined!")

✅ SIMPLIFIED Event detector class defined!


## 6. Portfolio Event-Driven Backtester

In [147]:
class PortfolioBacktester:
    """
    Event-driven portfolio backtester
    Tracks daily capital changes and generates performance metrics
    
    --- SIMPLIFIED VERSION ---
    - No dynamic/regime-based sizing or stops
    - No filter logic (RSI, ADX, BBW)
    - No special profit targets
    - No regime-change failsafes
    """
    
    def __init__(self, config: StrategyConfig, stock_data: Dict[str, pd.DataFrame], vix_data: pd.DataFrame):
        self.config = config
        self.stock_data = stock_data
        self.symbols = list(stock_data.keys())
        self.vix_data = vix_data
        
        # Portfolio state
        self.capital = config.initial_capital
        self.initial_capital = config.initial_capital
        self.positions: Dict[str, Position] = {}
        self.trades: List[Trade] = []
        self.event_log: List[Event] = []
        
        # Daily tracking
        self.daily_capital: List[Dict] = []
        self.daily_positions: List[Dict] = []
        # --- SIMPLIFIED: Changed default regime ---
        self.current_regime = MarketRegime.MEAN_REVERSION
        
        self.trading_dates = self.vix_data.index
        
        # Event detection
        self.event_detector = EventDetector(config)
        self.previous_data: Dict[str, pd.Series] = {}

    def get_market_regime(self, date: datetime) -> MarketRegime:
        """Get the market regime for a given date based on VIX"""
        if date in self.vix_data.index:
            
            vix_value_or_series = self.vix_data.loc[date, 'VIX_MA']
            
            if isinstance(vix_value_or_series, pd.Series):
                vix_value = vix_value_or_series.iloc[0]
            else:
                vix_value = vix_value_or_series
            
            # --- START OF SIMPLIFICATION ---
            if vix_value > self.config.vix_regime_threshold:
                return MarketRegime.TREND_FOLLOWING
            else:
                return MarketRegime.MEAN_REVERSION
            # --- END OF SIMPLIFICATION ---
        
        # If date is not in VIX data (e.g., holiday), use the last known regime
        return self.current_regime
    
    def calculate_position_size(self, symbol: str, price: float, atr_pct: float) -> float:
        """
        Calculate number of shares to buy.
        
        This CORRECTED version respects two different limits:
        1. Risk Limit (risk_per_trade): How much we are willing to lose.
        2. Allocation Limit (max_position_size): The max capital to put in one trade.
        
        The final position size will be the MINIMUM of these two calculations.
        """
        
        # --- 1. Calculate size based on RISK (Risk-Based Sizing) ---
        risk_amount_per_trade = self.capital * self.config.risk_per_trade
        
        # Use ATR-based stop distance for a dynamic risk calculation
        stop_distance_pct = max(atr_pct * 2, self.config.stop_loss_pct)
        dollar_risk_per_share = price * stop_distance_pct
        
        if dollar_risk_per_share == 0:
            return 0  # Avoid division by zero
            
        risk_based_shares = risk_amount_per_trade / dollar_risk_per_share
        
        # --- 2. Calculate size based on ALLOCATION (Allocation-Based Sizing) ---
        # This is the 'max_position_size' you set in your config.
        max_capital_to_allocate = self.capital * self.config.max_position_size
        allocation_based_shares = max_capital_to_allocate / price

        # --- 3. Final Size: Take the SMALLER of the two ---
        # We will never risk more than 'risk_per_trade'
        # AND we will never allocate more than 'max_position_size'.
        shares = max(risk_based_shares, allocation_based_shares)
        
        # --- 4. (Optional but good) Check against MINIMUM size ---
        # min_capital_to_allocate = self.capital * self.config.min_position_size
        # min_shares = min_capital_to_allocate / price
        
        # if shares < min_shares:
        #     return 0  # The trade is too small to be worth it
        
        return shares
    
    def calculate_stop_levels(self, price: float, atr_pct: float) -> Tuple[float, float]:
        """
        Calculate stop loss and trailing stop levels.
        SIMPLIFIED: No regime adjustment.
        """
        
        # --- START OF SIMPLIFICATION ---
        # Calculate ATR-based stops without regime multipliers
        stop_distance = max(atr_pct * 2, self.config.stop_loss_pct)
        trailing_distance = max(atr_pct * 1.5, self.config.trailing_stop_pct)
        
        stop_loss = price * (1 - stop_distance)
        trailing_stop = price * (1 - trailing_distance)
        # --- END OF SIMPLIFICATION ---
        
        return stop_loss, trailing_stop
    
    def open_position(self, symbol: str, price: float, timestamp: datetime, atr_pct: float):
        """Open a new position"""
        current_regime = self.get_market_regime(timestamp)
        
        # --- SIMPLIFIED: Removed 'regime' argument ---
        shares = self.calculate_position_size(symbol, price, atr_pct)
        position_cost = shares * price
        
        # --- SIMPLIFIED: Removed 'regime' argument ---
        stop_loss, trailing_stop = self.calculate_stop_levels(price, atr_pct)
        
        transaction_fee = position_cost * self.config.transaction_cost
        total_cost = position_cost + transaction_fee
        
        if total_cost > self.capital or shares <= 0:
            return  # Not enough capital or zero shares
        
        self.capital -= total_cost
        
        position = Position(
            symbol=symbol,
            side='LONG',
            entry_price=price,
            entry_time=timestamp,
            shares=shares,
            stop_loss=stop_loss,
            trailing_stop=trailing_stop,
            highest_price=price,
            regime=current_regime # We still tag the position with the regime it was opened in
        )
        
        self.positions[symbol] = position
    
    def can_open_position(self) -> bool:
        """Check if we can open a new position"""
        return len(self.positions) < self.config.max_positions
    
    def handle_mean_reversion_events(self, event: Event, current_data: pd.Series):
        """Handle mean reversion strategy events"""
        symbol = event.symbol
        
        # Entry conditions
        if (event.event_type == EventType.BB_LOWER_CROSS and symbol not in self.positions and current_data['Close'] > current_data['SMA_200']):
            # --- START OF SIMPLIFICATION (FILTERS REMOVED) ---
            if self.can_open_position():
                self.open_position(symbol, event.data['price'], event.timestamp, current_data['ATR_Pct'])
            # --- END OF SIMPLIFICATION ---
                
        # Exit conditions for longs
        elif symbol in self.positions:
            position = self.positions[symbol]
            if position.side == 'LONG':
                # Take profit at middle band
                if event.event_type == EventType.BB_MIDDLE_CROSS_UP:
                    self.close_position(symbol, event.data['price'], event.timestamp, "Target Reached - Middle Band")
                # --- 'ADX_STRONG_TREND' EXIT REMOVED ---
    
    def handle_trend_following_events(self, event: Event, current_data: pd.Series):
        """Handle trend following strategy events"""
        symbol = event.symbol
        
        # Entry conditions
        if event.event_type == EventType.GOLDEN_CROSS and symbol not in self.positions:
            # --- START OF SIMPLIFICATION (FILTERS REMOVED) ---
            if self.can_open_position():
                self.open_position(symbol, event.data['price'], event.timestamp, current_data['ATR_Pct'])
            # --- END OF SIMPLIFICATION ---
                
        # Exit conditions
        elif symbol in self.positions:
            position = self.positions[symbol]
            if position.side == 'LONG':
                if event.event_type == EventType.DEATH_CROSS:
                    self.close_position(symbol, event.data['price'], event.timestamp, "Exit - Death Cross")
                # --- 'ADX_WEAK_TREND' EXIT REMOVED ---
    
    def handle_regime_change(self, event: Event):
        """Handle regime change events"""
        new_regime = MarketRegime(event.data['new_regime'])
        
        # --- START OF SIMPLIFICATION (FAILSAFE REMOVED) ---
        # All logic for closing counter-trend positions has been removed
        # The only thing we do is update the current regime.
        # --- END OF SIMPLIFICATION ---
        
        # Update current regime
        self.current_regime = new_regime
    
    # --- 'check_profit_target' FUNCTION ENTIRELY REMOVED ---
    
    def handle_event(self, event: Event):
        """Handle a market event"""
        self.event_log.append(event)
        
        if event.event_type == EventType.REGIME_CHANGE:
            self.handle_regime_change(event)
            return 
        
        symbol = event.symbol
        
        if event.timestamp not in self.stock_data[symbol].index:
            return
            
        current_data = self.stock_data[symbol].loc[event.timestamp]

        # Apply regime-specific strategies
        # --- SIMPLIFIED: Updated to new Regime Enum ---
        if self.current_regime == MarketRegime.TREND_FOLLOWING:
            # Use trend following in high volatility
            self.handle_trend_following_events(event, current_data)
        else:
            # Use mean reversion in low/normal volatility
            self.handle_mean_reversion_events(event, current_data)
        
        # --- 'check_profit_target' CALL REMOVED ---

    def check_stops(self, symbol: str, current_price: float, timestamp: datetime):
        """Check if stops are hit for a position"""
        if symbol not in self.positions:
            return
        
        position = self.positions[symbol]
        
        # Update highest price and trailing stop
        if current_price > position.highest_price:
            position.highest_price = current_price
            # --- START OF SIMPLIFICATION ---
            # Removed regime-based logic. Trailing stop is calculated
            # the same way for all trades.
            position.trailing_stop = current_price * (1 - self.config.trailing_stop_pct)
            # --- END OF SIMPLIFICATION ---
        
        # Check stop loss
        if current_price <= position.stop_loss:
            self.close_position(symbol, current_price, timestamp, "Stop Loss")
        # Check trailing stop
        elif current_price <= position.trailing_stop:
            self.close_position(symbol, current_price, timestamp, "Trailing Stop")
            
    def calculate_portfolio_value(self, date: datetime) -> float:
        """Calculate total portfolio value (cash + open positions)"""
        market_value = 0.0
        for symbol, position in self.positions.items():
            if date in self.stock_data[symbol].index:
                current_price = self.stock_data[symbol].loc[date, 'Close']
                market_value += position.shares * current_price
            else:
                if symbol in self.previous_data:
                    market_value += position.shares * self.previous_data[symbol]['Close']
                else:
                    market_value += position.shares * position.entry_price
        
        return self.capital + market_value

    def close_position(self, symbol: str, price: float, timestamp: datetime, reason: str):
        """Close an open position and record the trade"""
        if symbol not in self.positions:
            return
        
        position = self.positions.pop(symbol)
        
        pnl = (price - position.entry_price) * position.shares
        pnl_pct = (price - position.entry_price) / position.entry_price
        
        transaction_fee = (price * position.shares) * self.config.transaction_cost
        total_pnl = pnl - transaction_fee
        
        self.capital += (price * position.shares) - transaction_fee
        
        trade = Trade(
            symbol=symbol,
            side=position.side,
            entry_time=position.entry_time,
            exit_time=timestamp,
            entry_price=position.entry_price,
            exit_price=price,
            shares=position.shares,
            pnl=total_pnl,
            pnl_pct=pnl_pct,
            reason=reason,
            regime=position.regime
        )
        self.trades.append(trade)

        self.event_log.append(Event(
            timestamp=timestamp,
            event_type=EventType.ORDER_FILLED,
            symbol=symbol,
            data={'side': 'SELL', 'price': price, 'shares': position.shares, 'reason': reason},
            priority=100
        ))
            
    def run(self):
        """Run the backtest"""
        print(f"\n{'='*60}")
        print("SIMPLIFIED EVENT-DRIVEN PORTFOLIO BACKTEST")
        print(f"{'='*60}")
        print(f"Period: {self.config.train_start} to {self.config.train_end}")
        print(f"Universe: {len(self.symbols)} stocks")
        print(f"Initial Capital: ${self.initial_capital:,.0f}")
        print(f"{'='*60}\n")
        
        # Initialize previous regime
        prev_regime = self.get_market_regime(self.trading_dates[0])
        self.current_regime = prev_regime # Ensure current_regime is set
        
        # Process each trading day
        for date in tqdm(self.trading_dates, desc="Processing days"):
            # Get current regime
            current_regime = self.get_market_regime(date)
            
            day_events = []
            
            # Check for regime change
            if current_regime != prev_regime:
                regime_event = Event(
                    timestamp=date,
                    event_type=EventType.REGIME_CHANGE,
                    symbol="MARKET", # Using "MARKET" is good practice
                    data={'new_regime': current_regime.value, 'old_regime': prev_regime.value},
                    priority=10
                )
                day_events.append(regime_event)
            
            for symbol in self.symbols:
                if date not in self.stock_data[symbol].index:
                    continue
                
                current_data = self.stock_data[symbol].loc[date]
                prev_data = self.previous_data.get(symbol)
                
                # Detect events
                if prev_data is not None: # Ensure we have previous data to detect events
                    events = self.event_detector.detect_events(symbol, current_data, prev_data,
                                                            current_regime, prev_regime)
                    day_events.extend(events)
                
                # --- DAILY CHECKS ---
                # Check stops (for open positions)
                self.check_stops(symbol, current_data['Close'], date)
                
                # --- 'check_profit_target' CALL REMOVED ---
                
                # Update previous data
                self.previous_data[symbol] = current_data
            
            # Sort events by priority
            day_events.sort(key=lambda e: e.priority, reverse=True)
            
            # Handle all events
            for event in day_events:
                self.handle_event(event)
            
            # Calculate daily portfolio value
            portfolio_value = self.calculate_portfolio_value(date)
            
            self.daily_capital.append({
                'date': date,
                'capital': self.capital,
                'portfolio_value': portfolio_value,
                'num_positions': len(self.positions),
                'invested': portfolio_value - self.capital,
                'regime': current_regime.value
            })
            
            # Update previous regime
            prev_regime = current_regime
        
        print(f"\n{'='*60}")
        print("BACKTEST COMPLETE")
        print(f"{'='*60}")
        print(f"Total Events: {len(self.event_log)}")
        print(f"Total Trades: {len(self.trades)}")
        final_value = self.daily_capital[-1]['portfolio_value']
        print(f"Final Portfolio Value: ${final_value:,.2f}")
        print(f"Total Return: {(final_value / self.initial_capital - 1) * 100:.2f}%")
        print(f"{'='*60}\n")

## 7. Performance Metrics Calculator

In [148]:
def calculate_portfolio_metrics(backtester: PortfolioBacktester, spy_data: Optional[pd.DataFrame] = None) -> Dict:
    """Calculate comprehensive portfolio performance metrics"""
    
    # Convert to DataFrame
    df = pd.DataFrame(backtester.daily_capital)
    if df.empty:
        print("⚠️ No daily capital data found. Cannot calculate metrics.")
        return {}
    
    df.set_index('date', inplace=True)
    
    # Add regime information
    df['regime'] = df.index.map(lambda x: backtester.get_market_regime(x).value)
    
    # Calculate returns
    df['daily_return'] = df['portfolio_value'].pct_change().fillna(0)
    df['cumulative_return'] = (df['portfolio_value'] / backtester.initial_capital) - 1
    
    # Drawdown
    df['peak'] = df['portfolio_value'].cummax()
    df['drawdown'] = (df['portfolio_value'] - df['peak']) / df['peak']
    
    # --- Overall Metrics ---
    total_return = (df['portfolio_value'].iloc[-1] / backtester.initial_capital) - 1
    n_days = len(df)
    n_years = n_days / 252
    
    annualized_return = 0.0
    if n_years > 0:
        annualized_return = (1 + total_return) ** (1 / n_years) - 1
        
    daily_vol = df['daily_return'].std()
    annualized_vol = daily_vol * np.sqrt(252)
    sharpe_ratio = annualized_return / annualized_vol if annualized_vol > 0 else 0
    max_drawdown = df['drawdown'].min()
    
    # Downside risk for Sortino
    downside_returns = df[df['daily_return'] < 0]['daily_return']
    downside_std = downside_returns.std() * np.sqrt(252)
    sortino_ratio = annualized_return / downside_std if downside_std > 0 else 0
    
    # Calmar Ratio
    calmar_ratio = annualized_return / abs(max_drawdown) if max_drawdown < 0 else 0
    
    # --- Overall Trade Statistics ---
    trades_df = pd.DataFrame([{
        'symbol': t.symbol,
        'pnl': t.pnl,
        'pnl_pct': t.pnl_pct,
        'reason': t.reason,
        'regime': t.regime.value
    } for t in backtester.trades])
    
    total_trades = 0
    win_rate = 0.0
    profit_factor = 0.0
    avg_win_pct = 0.0
    avg_loss_pct = 0.0
    
    if len(trades_df) > 0:
        total_trades = len(trades_df)
        wins = trades_df[trades_df['pnl'] > 0]
        losses = trades_df[trades_df['pnl'] < 0]
        
        win_rate = len(wins) / total_trades if total_trades > 0 else 0.0
        avg_win_pct = wins['pnl_pct'].mean() if len(wins) > 0 else 0.0
        avg_loss_pct = losses['pnl_pct'].mean() if len(losses) > 0 else 0.0
        
        total_profit = wins['pnl'].sum()
        total_loss = abs(losses['pnl'].sum())
        profit_factor = total_profit / total_loss if total_loss > 0 else float('inf')

    
    # --- Regime-specific Performance ---
    regime_metrics = {}
    for regime in MarketRegime:
        regime_data = df[df['regime'] == regime.value]
        if len(regime_data) > 0:
            regime_return = regime_data['daily_return'].mean() * 252  # Annualized
            regime_vol = regime_data['daily_return'].std() * np.sqrt(252)
            regime_sharpe = regime_return / regime_vol if regime_vol > 0 else 0
            regime_drawdown = regime_data['drawdown'].min()
            
            regime_metrics[regime.value] = {
                'days': len(regime_data),
                'annualized_return': regime_return,
                'annualized_vol': regime_vol,
                'sharpe_ratio': regime_sharpe,
                'max_drawdown': regime_drawdown,
                'pct_time': len(regime_data) / len(df) * 100
            }
    
    # --- Regime-specific Trade Stats ---
    regime_trade_stats = {}
    if len(trades_df) > 0:
        for regime in MarketRegime:
            regime_trades = trades_df[trades_df['regime'] == regime.value]
            if len(regime_trades) > 0:
                r_win_rate = (regime_trades['pnl'] > 0).sum() / len(regime_trades)
                r_avg_win = regime_trades[regime_trades['pnl'] > 0]['pnl_pct'].mean() if (regime_trades['pnl'] > 0).any() else 0
                r_avg_loss = regime_trades[regime_trades['pnl'] < 0]['pnl_pct'].mean() if (regime_trades['pnl'] < 0).any() else 0
                r_profit_factor = abs(regime_trades[regime_trades['pnl'] > 0]['pnl'].sum() / 
                                 regime_trades[regime_trades['pnl'] < 0]['pnl'].sum()) if (regime_trades['pnl'] < 0).any() else float('inf')
                
                regime_trade_stats[regime.value] = {
                    'total_trades': len(regime_trades),
                    'win_rate': r_win_rate,
                    'avg_win': r_avg_win,
                    'avg_loss': r_avg_loss,
                    'profit_factor': r_profit_factor
                }
    
    # --- START OF ADDED SECTION: BENCHMARK METRICS ---
    benchmark_metrics = {}
    if spy_data is not None:
        try:
            # Align benchmark data to our backtest dates
            bench_df = spy_data[['Close']].reindex(df.index).fillna(method='ffill')
            bench_df = bench_df.rename(columns={'Close': 'benchmark_close'})
            
            # Calculate benchmark portfolio value (buy-and-hold)
            bench_df['daily_return'] = bench_df['benchmark_close'].pct_change().fillna(0)
            bench_df['benchmark_value'] = backtester.initial_capital * (1 + bench_df['daily_return']).cumprod()
            
            # Add to the main daily_data df for plotting
            df['benchmark_value'] = bench_df['benchmark_value']

            # Calculate benchmark summary stats
            b_total_return = (bench_df['benchmark_value'].iloc[-1] / bench_df['benchmark_value'].iloc[0]) - 1
            b_annualized_return = (1 + b_total_return) ** (1 / n_years) - 1 if n_years > 0 else 0.0
            
            b_daily_vol = bench_df['daily_return'].std()
            b_annualized_vol = b_daily_vol * np.sqrt(252)
            b_sharpe_ratio = b_annualized_return / b_annualized_vol if b_annualized_vol > 0 else 0
            
            b_peak = bench_df['benchmark_value'].cummax()
            b_drawdown = (bench_df['benchmark_value'] - b_peak) / b_peak
            b_max_drawdown = b_drawdown.min()

            benchmark_metrics = {
                'total_return': b_total_return,
                'annualized_return': b_annualized_return,
                'annualized_volatility': b_annualized_vol,
                'sharpe_ratio': b_sharpe_ratio,
                'max_drawdown': b_max_drawdown,
                'final_value': bench_df['benchmark_value'].iloc[-1]
            }
        except Exception as e:
            print(f"⚠️ Failed to calculate benchmark metrics: {e}")
            benchmark_metrics = {} # Ensure it's an empty dict
    # --- END OF ADDED SECTION ---
    # --- Compile final dictionary ---
    return {
        'overall': {
            'total_return': total_return,
            'annualized_return': annualized_return,
            'annualized_volatility': annualized_vol,
            'sharpe_ratio': sharpe_ratio,
            'sortino_ratio': sortino_ratio,  # <-- ADDED
            'calmar_ratio': calmar_ratio,    # <-- ADDED
            'max_drawdown': max_drawdown,
            'total_trades': total_trades,    # <-- ADDED
            'win_rate': win_rate,            # <-- ADDED
            'profit_factor': profit_factor,  # <-- ADDED
            'avg_win_pct': avg_win_pct,      # <-- ADDED
            'avg_loss_pct': avg_loss_pct,    # <-- ADDED
            'total_events': len(backtester.event_log),
            'final_value': df['portfolio_value'].iloc[-1]
        },
        'regime_metrics': regime_metrics,
        'benchmark': benchmark_metrics,
        'regime_trade_stats': regime_trade_stats,
        'daily_data': df
    }

## 8. Visualization Functions

In [149]:
def plot_portfolio_performance(metrics: Dict):
    """Plot comprehensive portfolio performance"""
    
    df = metrics['daily_data']
    
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))
    
    # 1. Portfolio Value
    axes[0].plot(df.index, df['portfolio_value'], label='Portfolio Value', color='green', linewidth=2)
    axes[0].axhline(y=df['portfolio_value'].iloc[0], color='gray', linestyle='--', alpha=0.5, label='Initial Capital')
    # --- START OF ADDED SECTION ---
    if 'benchmark_value' in df.columns:
        axes[0].plot(df.index, df['benchmark_value'], 
                     label='SPY Benchmark', 
                     color='gray', 
                     linestyle='--', 
                     linewidth=2,
                     alpha=0.8)
    # --- END OF ADDED SECTION ---
    axes[0].set_title('Portfolio Value Over Time', fontsize=14, fontweight='bold')
    axes[0].set_ylabel('Portfolio Value ($)')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    axes[0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
    
    # 2. Drawdown
    axes[1].fill_between(df.index, 0, df['drawdown'] * 100, color='red', alpha=0.3)
    axes[1].axhline(y=-15, color='darkred', linestyle='--', linewidth=2, label='15% Kill Switch')
    axes[1].set_title('Portfolio Drawdown', fontsize=14, fontweight='bold')
    axes[1].set_ylabel('Drawdown (%)')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    # 3. Number of Positions
    axes[2].plot(df.index, df['num_positions'], label='Active Positions', color='purple', linewidth=2)
    axes[2].fill_between(df.index, 0, df['num_positions'], alpha=0.3, color='purple')
    axes[2].set_title('Number of Active Positions', fontsize=14, fontweight='bold')
    axes[2].set_ylabel('Positions')
    axes[2].set_xlabel('Date')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

def print_performance_summary(metrics: Dict):
    """Print formatted performance summary"""
    
    if not metrics or 'overall' not in metrics or not metrics['overall']:
        print("No metrics to display.")
        return
        
    overall = metrics['overall']
    benchmark = metrics.get('benchmark') # Use .get() to safely check if it exists
    
    print(f"\n{'='*80}")
    print("PORTFOLIO PERFORMANCE SUMMARY")
    print(f"{'='*80}\n")
    
    # Check if benchmark data is available
    if benchmark:
        print(f"{'Metric':<30} {'Strategy':>20} {'Benchmark (SPY)':>20}")
        print(f"{'-'*30:<30} {'-'*20:>20} {'-'*20:>20}")

        print("\nReturns:")
        print(f"{'Total Return':<30} {overall['total_return']*100:>19.2f}% {benchmark['total_return']*100:>19.2f}%")
        print(f"{'Annualized Return':<30} {overall['annualized_return']*100:>19.2f}% {benchmark['annualized_return']*100:>19.2f}%")
        print(f"{'Final Portfolio Value':<30} ${overall['final_value']:>19,.0f} ${benchmark['final_value']:>19,.0f}")
        
        print(f"\nRisk Metrics:")
        print(f"{'Annualized Volatility':<30} {overall['annualized_volatility']*100:>19.2f}% {benchmark['annualized_volatility']*100:>19.2f}%")
        print(f"{'Maximum Drawdown':<30} {overall['max_drawdown']*100:>19.2f}% {benchmark['max_drawdown']*100:>19.2f}%")
        
        print(f"\nRisk-Adjusted Returns:")
        print(f"{'Sharpe Ratio':<30} {overall['sharpe_ratio']:>20.2f} {benchmark['sharpe_ratio']:>20.2f}")
        print(f"{'Sortino Ratio':<30} {overall['sortino_ratio']:>20.2f} {'N/A':>20}")
        print(f"{'Calmar Ratio':<30} {overall['calmar_ratio']:>20.2f} {'N/A':>20}")
        
        print(f"\nTrading Statistics:")
        print(f"{'Total Trades':<30} {overall['total_trades']:>20} {'N/A':>20}")
        print(f"{'Total Events':<30} {overall['total_events']:>20} {'N/A':>20}")
        print(f"{'Win Rate':<30} {overall['win_rate']*100:>19.2f}% {'N/A':>20}")
        print(f"{'Profit Factor':<30} {overall['profit_factor']:>20.2f} {'N/A':>20}")
        print(f"{'Avg Win':<30} {overall['avg_win_pct']*100:>19.2f}% {'N/A':>20}")
        print(f"{'Avg Loss':<30} {overall['avg_loss_pct']*100:>19.2f}% {'N/A':>20}")

    else:
        # Fallback to old format if no benchmark
        print("Returns:")
        print(f"  Total Return.................. {overall['total_return']*100:>10.2f}%")
        print(f"  Annualized Return............. {overall['annualized_return']*100:>10.2f}%")
        print(f"  Final Portfolio Value......... ${overall['final_value']:>10,.0f}")
        
        print(f"\nRisk Metrics:")
        print(f"  Annualized Volatility......... {overall['annualized_volatility']*100:>10.2f}%")
        print(f"  Maximum Drawdown.............. {overall['max_drawdown']*100:>10.2f}%")
        
        print(f"\nRisk-Adjusted Returns:")
        print(f"  Sharpe Ratio.................. {overall['sharpe_ratio']:>10.2f}")
        print(f"  Sortino Ratio................. {overall['sortino_ratio']:>10.2f}")
        print(f"  Calmar Ratio.................. {overall['calmar_ratio']:>10.2f}")
        
        print(f"\nTrading Statistics:")
        print(f"  Total Trades.................. {overall['total_trades']:>10}")
        print(f"  Total Events.................. {overall['total_events']:>10}")
        print(f"  Win Rate...................... {overall['win_rate']*100:>10.2f}%")
        print(f"  Profit Factor................. {overall['profit_factor']:>10.2f}")
        print(f"  Avg Win....................... {overall['avg_win_pct']*100:>10.2f}%")
        print(f"  Avg Loss...................... {overall['avg_loss_pct']*100:>10.2f}%")
    
    # --- END OF MODIFIED SECTION ---
    
    print(f"\n{'='*80}\n")

print("✅ (Updated) Summary printing function defined!")

✅ (Updated) Summary printing function defined!


## 9. Main Backtest Function

In [150]:
def run_backtest(start_date: str, 
                 end_date: str, 
                 stock_list: List[str],
                 initial_capital: float = 500000.0,
                 transaction_cost: float = 0.0,
                 leverage: float = 0.0,
                 benchmark_ticker: str='SPY',
                 **other_params) -> Dict:
    """
    Run complete backtest on portfolio of stocks
    
    Args:
        start_date: Start date (YYYY-MM-DD)
        end_date: End date (YYYY-MM-DD)
        stock_list: List of stock symbols
        initial_capital: Starting capital
        transaction_cost: Transaction cost as decimal (0 = 0%)
        leverage: Leverage multiplier (0 = no leverage)
        **other_params: Additional strategy parameters
    
    Returns:
        Dictionary with results and metrics
    """
    
    print(f"\n{'='*60}")
    print("DOWNLOADING DATA")
    print(f"{'='*60}")
    print(f"Period: {start_date} to {end_date}")
    print(f"Stocks: {len(stock_list)} symbols")
    print(f"Benchmark: {benchmark_ticker}") # <-- ADDED
    
    # --- START OF ADDED SECTION ---
    # Download Benchmark Data
    try:
        spy_data = yf.download(benchmark_ticker, 
                              start=start_date, 
                              end=end_date, 
                              progress=False)
        spy_data.columns = spy_data.columns.get_level_values(0)
        spy_data.index = spy_data.index.tz_localize(None)
        print(spy_data)
        if spy_data.empty:
            raise ValueError(f"{benchmark_ticker} download failed")
        print(f"✅ Successfully loaded benchmark data for {benchmark_ticker}")
    except Exception as e:
        print(f"  ⚠️  Failed to download benchmark {benchmark_ticker}: {e}")
        print("  Proceeding without benchmark comparison.")
        spy_data = None
    # --- END OF ADDED SECTION ---
    
    # Download data for all stocks
    stock_data = {}
    config = StrategyConfig(
        train_start=start_date,
        train_end=end_date,
        initial_capital=initial_capital,
        transaction_cost=transaction_cost,
        leverage=leverage,
        **other_params
    )
    
    for symbol in tqdm(stock_list, desc="Downloading"):
        try:
            ticker = yf.Ticker(symbol)
            df = ticker.history(start=start_date, end=end_date)
            df.columns = df.columns.get_level_values(0)
            # --- START OF FIX ---
            # Strip timezone information to make it compatible with VIX data
            df.index = df.index.tz_localize(None)
            # --- END OF FIX ---
            
            if len(df) < 100:  # Skip if insufficient data
                continue
            
            # Calculate indicators
            df_with_indicators = calculate_indicators(df, config)
            stock_data[symbol] = df_with_indicators
            
        except Exception as e:
            print(f"  ⚠️  Failed to download {symbol}: {e}")
            continue
        
    vix_data = load_vix_data(config)
    print("\n" + "="*40)
    print("--- DEBUG: VIX DATA & REGIMES ---")
    print(vix_data[['VIX_MA', 'Regime']].tail())
    print("\nRegime Counts:")
    print(vix_data['Regime'].value_counts(normalize=True) * 100)
    print("="*40)
    
    print("\n" + "="*40)
    print("--- DEBUG: SAMPLE STOCK DATA (AAPL) ---")
    if 'AAPL' in stock_data:
        # Print the last 5 rows of AAPL's indicator data
        print(stock_data['AAPL'].tail())
    else:
        print("AAPL data not found, printing first available stock.")
        first_stock = list(stock_data.keys())[0]
        print(stock_data[first_stock].tail())
    print("="*40 + "\n")
    # --- END OF DEBUGGING (STEP 1) ---
    
    print(f"\n✅ Successfully loaded {len(stock_data)} stocks\n")
    
    # Run backtest
    backtester = PortfolioBacktester(config, stock_data, vix_data)
    backtester.run()
    
    # --- START OF DEBUGGING (STEP 2) ---
    print("\n" + "="*40)
    print("--- DEBUG: EVENT LOG SUMMARY ---")
    from collections import Counter
    event_counts = Counter(e.event_type.value for e in backtester.event_log)
    
    if not event_counts:
        print("⚠️ CRITICAL: No events were generated at all!")
    else:
        print("Total events generated during backtest:")
        for event_type, count in event_counts.most_common():
            # Pad the event type string to align the counts
            print(f"  {event_type:<25} {count}")
    print("="*40 + "\n")
    # --- END OF DEBUGGING (STEP 2) ---
    # Calculate metrics
    metrics = calculate_portfolio_metrics(backtester, spy_data)
    
    # Visualize
    plot_portfolio_performance(metrics)
    print_performance_summary(metrics)
    
    return {
        'metrics': metrics,
        'backtester': backtester,
        'config': config,
        'daily_capital': backtester.daily_capital,
        'trades': backtester.trades
    }

print("✅ Main backtest function defined!")

✅ Main backtest function defined!


In [164]:
import itertools

def download_raw_stock_data(stock_list: List[str], start_date: str, end_date: str) -> Dict[str, pd.DataFrame]:
    """
    Downloads and caches raw stock data (Open, High, Low, Close, Volume)
    without any indicators.
    """
    print(f"\n{'='*60}")
    print("DOWNLOADING RAW STOCK DATA")
    print(f"{'='*60}")
    
    raw_stock_data = {}
    
    for symbol in tqdm(stock_list, desc="Downloading Raw Data"):
        try:
            ticker = yf.Ticker(symbol)
            df = ticker.history(start=start_date, end=end_date)
            
            # --- START OF FIX ---
            # Check if the DataFrame is empty (i.e., download failed)
            # This must be done BEFORE any processing.
            if df.empty:
                print(f"  ⚠️  No data returned for {symbol}. Skipping.")
                continue
            # --- END OF FIX ---

            # 1. Strip timezone (if it exists)
            # This is now safe because we know df is not empty
            if df.index.tz is not None:
                df.index = df.index.tz_localize(None)
            
            # 2. Get base columns
            # The .get_level_values(0) calls were not needed for ticker.history()
            df = df[['Open', 'High', 'Low', 'Close', 'Volume']]
            
            if len(df) < 100:  # Skip if insufficient data
                print(f"  ⚠️  Insufficient data for {symbol} (found {len(df)} bars). Skipping.")
                continue
                
            raw_stock_data[symbol] = df
            
        except Exception as e:
            # This will now catch any other unexpected errors
            print(f"  ⚠️  Failed to process {symbol}: {e}")
            continue
    
    print(f"\n✅ Successfully loaded raw data for {len(raw_stock_data)} stocks\n")
    return raw_stock_data

# Note: The rest of your code (run_parameter_grid_search) looks correct 
# and does not need to be changed based on this error.


def run_parameter_grid_search(raw_stock_data: Dict[str, pd.DataFrame], 
                              vix_data: pd.DataFrame, # We still take the base VIX data
                              base_config_dict: Dict,
                              param_grid: Dict) -> pd.DataFrame:
    """
    Runs a "Grid Search" to test every combination of parameters.
    
    This is the CORRECTED version that recalculates
    *both* stock indicators and VIX data as needed.
    """
    print(f"\n{'='*60}")
    print("STARTING CORRECTED PARAMETER GRID SEARCH")
    print(f"(This will be slow as it recalculates indicators...)")
    print(f"{'='*60}\n")
    
    # 1. Get all combinations from the grid
    keys, value_lists = zip(*param_grid.items())
    all_combinations = [dict(zip(keys, combo)) for combo in itertools.product(*value_lists)]
    
    total_runs = len(all_combinations)
    print(f"Total combinations to test: {total_runs}")
    
    results_log = []

    # 2. Loop through each combination
    for i, params_to_test in enumerate(tqdm(all_combinations, desc="Grid Search")):
        
        # Create a copy of the base config and update it
        current_config_dict = base_config_dict.copy()
        current_config_dict.update(params_to_test)
        
        try:
            # 3. Create the new config object
            test_config = StrategyConfig(**current_config_dict)
            
            # Simple check
            if hasattr(test_config, 'ema_fast') and hasattr(test_config, 'ema_slow'):
                if test_config.ema_fast >= test_config.ema_slow:
                    continue

            # 4. --- START OF FIX ---
            # Check if VIX parameters are being tested. If so,
            # we MUST reload the VIX data with the new config.
            vix_params_changed = any(k in params_to_test for k in 
                                     ['vix_smoothing_period', 'vix_regime_threshold'])
            
            if vix_params_changed:
                current_vix_data = load_vix_data(test_config)
            else:
                current_vix_data = vix_data # Use the cached, pre-loaded version
            # --- END OF FIX ---

            # 5. Recalculate all stock indicators for this specific config
            stock_data_with_indicators = {}
            for symbol, raw_df in raw_stock_data.items():
                df_with_indicators = calculate_indicators(raw_df.copy(), test_config)
                stock_data_with_indicators[symbol] = df_with_indicators

            # 6. Run the backtest with the newly calculated data
            backtester = PortfolioBacktester(
                test_config, 
                stock_data_with_indicators, 
                current_vix_data  # Use the (potentially new) VIX data
            )
            
            # Suppress the backtester's own print statements
            # You can comment out the .run() call in the next line 
            # and uncomment the one after if you want to see the full output
            
            # --- Quiet mode ---
            import io
            import sys
            old_stdout = sys.stdout
            sys.stdout = io.StringIO()
            try:
                backtester.run()
            finally:
                sys.stdout = old_stdout
            
            # --- Verbose mode (uncomment to see all output) ---
            # backtester.run() 
            
            metrics = calculate_portfolio_metrics(backtester)
            
            if 'overall' not in metrics or not metrics['overall']:
                continue
            
            # 7. Store the results
            run_result = {
                'sharpe': metrics['overall']['sharpe_ratio'],
                'return': metrics['overall']['annualized_return'],
                'drawdown': metrics['overall']['max_drawdown'],
                'trades': metrics['overall']['total_trades']
            }
            # Add the parameters that were tested
            run_result.update(params_to_test)
            
            results_log.append(run_result)

        except Exception as e:
            # print(f"Test {i+1} failed with params {params_to_test}: {e}")
            continue # Fail silently to not clog the tqdm bar

    # 8. Return a DataFrame for easy analysis
    results_df = pd.DataFrame(results_log)
    
    print(f"\n{'='*60}")
    print("GRID SEARCH COMPLETE")
    print(f"{'='*60}\n")
    
    if results_df.empty:
        print("⚠️ No results were successfully logged. Check parameters.")
    else:
        # Show the best results
        print("--- Top 10 Results by Sharpe Ratio ---")
        print(results_df.sort_values(by='sharpe', ascending=False).head(10))
    
    return results_df

In [165]:
# --- 1. Define base parameters ---
start_date = "2010-01-01"
end_date = "2019-12-31"
stock_list = SPY_STOCKS

# --- 2. Download RAW stock data (no indicators) ---
# This is a one-time download.
raw_stock_data = download_raw_stock_data(stock_list, start_date, end_date)

# --- 3. Create a 'base_config_dict' ---
# This dictionary will hold all default values,
# which will then be overridden by our param_grid.

# Create a default config object to get all default values
default_config = StrategyConfig()

# Convert it to a dictionary
base_config_dict = {
    'ema_fast': default_config.ema_fast,
    'ema_slow': default_config.ema_slow,
    'atr_period': default_config.atr_period,
    'bb_period': default_config.bb_period,
    'bb_std': default_config.bb_std,
    'vix_regime_threshold': default_config.vix_regime_threshold,
    'vix_smoothing_period': default_config.vix_smoothing_period,
    'stop_loss_pct': default_config.stop_loss_pct,
    'trailing_stop_pct': default_config.trailing_stop_pct,
    'max_portfolio_drawdown': default_config.max_portfolio_drawdown,
    'risk_per_trade': default_config.risk_per_trade,
    'min_position_size': default_config.min_position_size,
    'max_position_size': default_config.max_position_size,
    'initial_capital': default_config.initial_capital,
    'transaction_cost': default_config.transaction_cost,
    'leverage': default_config.leverage,
    'max_positions': default_config.max_positions,
    'train_start': start_date,
    'train_end': end_date
}

# --- 4. Load the base VIX data (to be used when VIX params aren't changing) ---
# We pass the *default_config* object here
base_vix_data = load_vix_data(default_config)

print("✅ Setup complete. Ready for grid search.")


DOWNLOADING RAW STOCK DATA


$NVDA: possibly delisted; no price data found  (1d 2010-01-01 -> 2019-12-31) (Yahoo error = "Request Failed")
$MSFT: possibly delisted; no price data found  (1d 2010-01-01 -> 2019-12-31) (Yahoo error = "Error occurred while retrieving timeseries, keys: [RedisKey [key=TF[1,YF_HIST][5970], cluster=finance]]")
$AVGO: possibly delisted; no price data found  (1d 2010-01-01 -> 2019-12-31) (Yahoo error = "Error occurred while retrieving timeseries, keys: [RedisKey [key=TF[1,YF_HIST,DV][89843], cluster=finance]]")


  ⚠️  No data returned for NVDA. Skipping.
  ⚠️  No data returned for MSFT. Skipping.
  ⚠️  No data returned for AVGO. Skipping.

✅ Successfully loaded raw data for 7 stocks

✅ Setup complete. Ready for grid search.


In [166]:
# # --- 1. Define the COMBINED parameter grid ---
# # This dictionary contains all parameters from all 3 tests.

# param_grid_combined = {
#     # --- From Grid 1: EMA ---
#     'ema_fast': [10, 15, 20],        # 3 values
#     'ema_slow': [40, 50, 60],        # 3 values
    
#     # --- From Grid 2: Bollinger Bands ---
#     'bb_std': [1.5, 2.0, 2.5],      # 3 values
    
#     # --- From Grid 3: VIX ---
#     'vix_regime_threshold': [25.0, 30.0, 35.0],  # 3 values
#     'vix_smoothing_period': [5, 10]              # 2 values
# }

# # Total runs = 3 * 3 * 3 * 3 * 2 = 162 runs
# # (This may take 10-20 minutes or more to run!)

# print(f"\n--- Starting COMBINED Grid Search: All Parameters ---")
# print(f"Total runs to execute: {3*3*3*3*2}")

# # --- 2. Run the grid search ---
# # We use the *exact same functions* as before.
# # The 'base_config_dict' provides the defaults for
# # any parameter we are *not* testing (like 'max_positions').

# combined_results_df = run_parameter_grid_search(
#     raw_stock_data=raw_stock_data,
#     vix_data=base_vix_data,
#     base_config_dict=base_config_dict,
#     param_grid=param_grid_combined
# )

# # --- 3. View the results ---
# print("\n--- Best Results from COMBINED Grid Search ---")
# print(combined_results_df.sort_values(by='sharpe', ascending=False).head(20))

## 10. FINAL NOTEBOOK CELL
---


DOWNLOADING DATA
Period: 2010-01-01 to 2019-12-31
Stocks: 10 symbols
Benchmark: SPY
Price            Close        High         Low        Open     Volume
Date                                                                 
2010-01-04   85.279228   85.324376   83.909704   84.556842  118944600
2010-01-05   85.504959   85.542586   84.918021   85.226543  111579900
2010-01-06   85.565155   85.775850   85.354460   85.422181  116074400
2010-01-07   85.926331   86.031678   85.166319   85.407114  131091100
2010-01-08   86.212288   86.249914   85.527529   85.700597  126402800
...                ...         ...         ...         ...        ...
2019-12-23  295.224060  295.619254  295.077005  295.564112   52990000
2019-12-24  295.233276  295.499787  294.929967  295.453844   20270000
2019-12-26  296.804871  296.814070  295.610089  295.619261   30911200
2019-12-27  296.731293  297.595222  296.198244  297.540080   42528800
2019-12-30  295.095367  296.951909  294.608261  296.814054   49729100

[251

Downloading:   0%|          | 0/10 [00:00<?, ?it/s]

$NVDA: possibly delisted; no price data found  (1d 2010-01-01 -> 2019-12-31) (Yahoo error = "Request Failed")
$MSFT: possibly delisted; no price data found  (1d 2010-01-01 -> 2019-12-31) (Yahoo error = "Error occurred while retrieving timeseries, keys: [RedisKey [key=TF[1,YF_HIST][5970], cluster=finance]]")


  ⚠️  Failed to download NVDA: 'Index' object has no attribute 'tz_localize'
  ⚠️  Failed to download MSFT: 'Index' object has no attribute 'tz_localize'


$AVGO: possibly delisted; no price data found  (1d 2010-01-01 -> 2019-12-31) (Yahoo error = "Error occurred while retrieving timeseries, keys: [RedisKey [key=TF[1,YF_HIST,DV][89843], cluster=finance]]")


  ⚠️  Failed to download AVGO: 'Index' object has no attribute 'tz_localize'

--- DEBUG: VIX DATA & REGIMES ---
Price       VIX_MA          Regime
Ticker                            
Date                              
2019-12-23  13.187  mean_reversion
2019-12-24  12.886  mean_reversion
2019-12-26  12.652  mean_reversion
2019-12-27  12.601  mean_reversion
2019-12-30  12.820  mean_reversion

Regime Counts:
Regime
mean_reversion     92.577813
trend_following     7.422187
Name: proportion, dtype: float64

--- DEBUG: SAMPLE STOCK DATA (AAPL) ---
                Close       High        Low     Volume   EMA_Fast   EMA_Slow  \
Date                                                                           
2019-12-23  68.589760  68.650138  67.713066   98572000  65.794239  62.650601   
2019-12-24  68.654991  68.804736  68.328954   48478800  66.066692  62.886067   
2019-12-26  70.017120  70.034028  68.758838   93121200  66.442923  63.165716   
2019-12-27  69.990540  70.997653  69.584799  14626600

Processing days:   0%|          | 0/2506 [00:00<?, ?it/s]


BACKTEST COMPLETE
Total Events: 4341
Total Trades: 354
Final Portfolio Value: $854,931.11
Total Return: 70.99%


--- DEBUG: EVENT LOG SUMMARY ---
Total events generated during backtest:
  bb_upper_cross            972
  bb_middle_cross_down      944
  bb_middle_cross_up        943
  bb_lower_cross            704
  order_filled              354
  golden_cross              163
  death_cross               159
  regime_change             102



PORTFOLIO PERFORMANCE SUMMARY

Metric                                     Strategy      Benchmark (SPY)
------------------------------ -------------------- --------------------

Returns:
Total Return                                 70.99%              245.09%
Annualized Return                             5.54%               13.26%
Final Portfolio Value          $            854,931 $          1,725,453

Risk Metrics:
Annualized Volatility                         7.79%               14.71%
Maximum Drawdown                            -17.32%              -19.35%

Risk-Adjusted Returns:
Sharpe Ratio                                   0.71                 0.90
Sortino Ratio                                  0.56                  N/A
Calmar Ratio                                   0.32                  N/A

Trading Statistics:
Total Trades                                    354                  N/A
Total Events                                   4341                  N/A
Win Rate              

In [167]:
# ============================================================================
# FINAL NOTEBOOK CELL
# ============================================================================

# Define parameters for the backtest
start_date = "2010-01-01"  # We will change this
end_date = "2019-12-31"    # We will change this
stock_list = SPY_STOCKS    # We will also change this

# Other parameters
other_variables = {
    'initial_capital': 500000.0,
    'transaction_cost': 0.0,
    'leverage': 0.0,
    'max_positions': 10,
    'max_position_size': 0.1,
    'stop_loss_pct': 0.05,
    'trailing_stop_pct': 0.015,
    'max_portfolio_drawdown': 0.15,
    'risk_per_trade': 0.02
}

# One function that runs the backtest
results = run_backtest(start_date=start_date, 
                       end_date=end_date,
                       stock_list=stock_list,
                       benchmark_ticker='SPY',
                       **other_variables)


In [168]:
# # 1. Get the RAW data *once*
# # (This assumes 'stock_list', 'start_date', 'end_date' are defined)
# raw_data_cache = download_raw_stock_data(stock_list, start_date, end_date)

# # 2. Define the grid of parameters you want to "tweak"
# param_grid_to_test = {
#     # --- Trend Following Params ---
#     'ema_fast': [10, 20],
#     'ema_slow': [50],
#     'adx_threshold': [20, 25],
#     'adx_period': [14, 20],

#     # --- Mean Reversion Params ---
#     'bb_period': [20, 30],
#     'bb_std': [2.0, 2.5],
#     'rsi_period': [10, 14],
    
#     # --- Regime Params ---
#     'vix_low_threshold': [15],
#     'vix_high_threshold': [20, 25, 30]
# }

# # 3. Calculate and print total combinations
# total_combinations = 1
# for key in param_grid_to_test:
#     total_combinations *= len(param_grid_to_test[key])
# print(f"--- Starting Grid Search ---")
# print(f"Total combinations to test: {total_combinations}") # This will be 2*1*2*2 * 2*2*2 * 2*2 = 512 runs


# # 4. Get the other variables from your main backtest run
# if 'results' in locals():
#     # We only need the VIX data from the main run
#     base_vix_data = results['backtester'].vix_data
    
#     # 5. Run the NEW grid search function
#     grid_search_results = run_parameter_grid_search(
#         raw_stock_data=raw_data_cache,    # <-- Use the new raw data
#         vix_data=base_vix_data,
#         base_config_dict=other_variables, # Your dict from the main run
#         param_grid=param_grid_to_test
#     )
    
#     # 6. (Optional) Show results sorted by Drawdown
#     print("\n\n--- Top 10 Results by Lowest Max Drawdown ---")
#     print(grid_search_results.sort_values(by='drawdown', ascending=False).head(10))

# else:
#     print("❌ Please run the main 'run_backtest()' cell first to create the 'results' variable!")

In [169]:
# ============================================================================
# BUY-AND-HOLD BENCHMARK CALCULATOR
# ============================================================================

def calculate_buy_and_hold(stock_list: List[str], 
                          start_date: str, 
                          end_date: str, 
                          initial_capital: float = 500000.0) -> Dict:
    """
    Calculate buy-and-hold returns for equal-weighted portfolio
    
    Strategy: Buy equal amounts of all stocks at start, hold until end
    """
    print(f"\n{'='*60}")
    print("CALCULATING BUY-AND-HOLD BENCHMARK")
    print(f"{'='*60}")
    
    portfolio_data = []
    
    # Download data for all stocks
    for symbol in tqdm(stock_list, desc="Downloading B&H data"):
        try:
            ticker = yf.Ticker(symbol)
            df = ticker.history(start=start_date, end=end_date)
            
            if len(df) < 100:
                continue
            
            portfolio_data.append({
                'symbol': symbol,
                'start_price': df['Close'].iloc[0],
                'end_price': df['Close'].iloc[-1],
                'data': df
            })
            
        except Exception as e:
            continue
    
    # Equal weight allocation
    capital_per_stock = initial_capital / len(portfolio_data)
    
    # Calculate daily portfolio values
    total_value = 0
    daily_values = {}
    
    for stock in portfolio_data:
        shares = capital_per_stock / stock['start_price']
        final_value = shares * stock['end_price']
        total_value += final_value
        
        # Track daily values
        for date, row in stock['data'].iterrows():
            if date not in daily_values:
                daily_values[date] = 0
            daily_values[date] += shares * row['Close']
    
    # Create daily portfolio dataframe
    df_daily = pd.DataFrame([
        {'date': date, 'portfolio_value': value}
        for date, value in sorted(daily_values.items())
    ])
    df_daily.set_index('date', inplace=True)
    
    # Calculate returns
    df_daily['daily_return'] = df_daily['portfolio_value'].pct_change()
    df_daily['cumulative_return'] = (df_daily['portfolio_value'] / initial_capital) - 1
    
    # Drawdown
    df_daily['peak'] = df_daily['portfolio_value'].cummax()
    df_daily['drawdown'] = (df_daily['portfolio_value'] - df_daily['peak']) / df_daily['peak']
    
    # Performance metrics
    total_return = (total_value / initial_capital) - 1
    n_days = len(df_daily)
    n_years = n_days / 252
    annualized_return = (1 + total_return) ** (1 / n_years) - 1
    
    daily_vol = df_daily['daily_return'].std()
    annualized_vol = daily_vol * np.sqrt(252)
    
    sharpe_ratio = annualized_return / annualized_vol if annualized_vol > 0 else 0
    
    downside_returns = df_daily['daily_return'][df_daily['daily_return'] < 0]
    downside_std = downside_returns.std() * np.sqrt(252) if len(downside_returns) > 0 else 0.0001
    sortino_ratio = annualized_return / downside_std if downside_std > 0 else 0
    
    max_drawdown = df_daily['drawdown'].min()
    calmar_ratio = annualized_return / abs(max_drawdown) if max_drawdown != 0 else 0
    
    print(f"\n✅ Buy-and-Hold calculated for {len(portfolio_data)} stocks")
    print(f"Final Value: ${total_value:,.2f}")
    print(f"Total Return: {total_return*100:.2f}%")
    
    return {
        'total_return': total_return,
        'annualized_return': annualized_return,
        'annualized_volatility': annualized_vol,
        'sharpe_ratio': sharpe_ratio,
        'sortino_ratio': sortino_ratio,
        'max_drawdown': max_drawdown,
        'calmar_ratio': calmar_ratio,
        'final_value': total_value,
        'daily_data': df_daily,
        'num_stocks': len(portfolio_data)
    }

print("✅ Buy-and-Hold benchmark function defined!")

✅ Buy-and-Hold benchmark function defined!


In [170]:
def compare_strategy_vs_benchmark(strategy_metrics: Dict, 
                                  bh_metrics: Dict,
                                  start_date: str,
                                  end_date: str):
    """
    Compare strategy performance vs buy-and-hold benchmark
    """
    
    # Extract data
    strategy_data = strategy_metrics['daily_data']
    bh_data = bh_metrics['daily_data']
    
    # Create comparison plots
    fig, axes = plt.subplots(3, 2, figsize=(18, 14))
    
    # 1. Portfolio Value Comparison
    axes[0, 0].plot(strategy_data.index, strategy_data['portfolio_value'], 
                    label='Strategy', color='green', linewidth=2)
    axes[0, 0].plot(bh_data.index, bh_data['portfolio_value'], 
                    label='Buy & Hold', color='blue', linewidth=2, alpha=0.7)
    
    # FIX: Access strategy_metrics['overall'] but bh_metrics directly
    axes[0, 0].axhline(y=strategy_metrics['overall']['final_value'], color='green', 
                       linestyle='--', alpha=0.3)
    axes[0, 0].axhline(y=bh_metrics['final_value'], color='blue',  # <-- FIXED
                       linestyle='--', alpha=0.3)
    
    axes[0, 0].set_title('Portfolio Value: Strategy vs Buy-and-Hold', 
                         fontsize=14, fontweight='bold')
    axes[0, 0].set_ylabel('Portfolio Value ($)')
    axes[0, 0].legend(loc='best')
    axes[0, 0].grid(True, alpha=0.3)
    axes[0, 0].yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'${x/1e6:.1f}M'))
    
    # ... (Plots 0,1 and 1,0 and 1,1 are fine as they only use 'daily_data') ...
    
    # 4. Rolling Sharpe Ratio (252-day window)
    strategy_rolling_sharpe = (strategy_data['daily_return'].rolling(252).mean() * 252) / \
                              (strategy_data['daily_return'].rolling(252).std() * np.sqrt(252))
    bh_rolling_sharpe = (bh_data['daily_return'].rolling(252).mean() * 252) / \
                        (bh_data['daily_return'].rolling(252).std() * np.sqrt(252))
    
    axes[1, 1].plot(strategy_data.index, strategy_rolling_sharpe, 
                    label='Strategy', color='green', linewidth=2)
    axes[1, 1].plot(bh_data.index, bh_rolling_sharpe, 
                    label='Buy & Hold', color='blue', linewidth=2, alpha=0.7)
    axes[1, 1].axhline(y=1.0, color='red', linestyle='--', linewidth=1, label='Sharpe = 1.0')
    axes[1, 1].set_title('Rolling Sharpe Ratio (1-Year)', fontsize=14, fontweight='bold')
    axes[1, 1].set_ylabel('Sharpe Ratio')
    axes[1, 1].legend(loc='best')
    axes[1, 1].grid(True, alpha=0.3)
    
    
    # 5. Performance Metrics Bar Chart
    # FIX: Access strategy_metrics['overall'] but bh_metrics directly
    metrics_comparison = {
        'Total Return (%)': [strategy_metrics['overall']['total_return'] * 100, 
                            bh_metrics['total_return'] * 100], # <-- FIXED
        'Ann. Return (%)': [strategy_metrics['overall']['annualized_return'] * 100, 
                           bh_metrics['annualized_return'] * 100], # <-- FIXED
        'Sharpe Ratio': [strategy_metrics['overall']['sharpe_ratio'], 
                        bh_metrics['sharpe_ratio']], # <-- FIXED
        'Sortino Ratio': [strategy_metrics['overall']['sortino_ratio'], 
                         bh_metrics['sortino_ratio']], # <-- FIXED
        'Calmar Ratio': [strategy_metrics['overall']['calmar_ratio'], 
                        bh_metrics['calmar_ratio']] # <-- FIXED
    }
    
    x = np.arange(len(metrics_comparison))
    width = 0.35
    
    strategy_values = [v[0] for v in metrics_comparison.values()]
    bh_values = [v[1] for v in metrics_comparison.values()]
    
    axes[2, 0].bar(x - width/2, strategy_values, width, label='Strategy', color='green', alpha=0.7)
    axes[2, 0].bar(x + width/2, bh_values, width, label='Buy & Hold', color='blue', alpha=0.7)
    axes[2, 0].set_title('Performance Metrics Comparison', fontsize=14, fontweight='bold')
    axes[2, 0].set_ylabel('Value')
    axes[2, 0].set_xticks(x)
    axes[2, 0].set_xticklabels(metrics_comparison.keys(), rotation=45, ha='right')
    axes[2, 0].legend(loc='best')
    axes[2, 0].grid(True, alpha=0.3, axis='y')
    axes[2, 0].axhline(y=0, color='black', linestyle='-', linewidth=0.5)
    
    # 6. Risk Metrics Bar Chart
    # FIX: Access strategy_metrics['overall'] but bh_metrics directly
    risk_comparison = {
        'Ann. Vol (%)': [strategy_metrics['overall']['annualized_volatility'] * 100, 
                        bh_metrics['annualized_volatility'] * 100], # <-- FIXED
        'Max DD (%)': [strategy_metrics['overall']['max_drawdown'] * 100, 
                      bh_metrics['max_drawdown'] * 100] # <-- FIXED
    }
    
    x_risk = np.arange(len(risk_comparison))
    strategy_risk = [v[0] for v in risk_comparison.values()]
    bh_risk = [v[1] for v in risk_comparison.values()]
    
    axes[2, 1].bar(x_risk - width/2, strategy_risk, width, label='Strategy', color='green', alpha=0.7)
    axes[2, 1].bar(x_risk + width/2, bh_risk, width, label='Buy & Hold', color='blue', alpha=0.7)
    axes[2, 1].set_title('Risk Metrics Comparison (Lower is Better)', fontsize=14, fontweight='bold')
    axes[2, 1].set_ylabel('Value (%)')
    axes[2, 1].set_xticks(x_risk)
    axes[2, 1].set_xticklabels(risk_comparison.keys())
    axes[2, 1].legend(loc='best')
    axes[2, 1].grid(True, alpha=0.3, axis='y')
    
    plt.tight_layout()
    plt.show()
    
    # Print comparison table
    print(f"\n{'='*80}")
    print(f"STRATEGY VS BUY-AND-HOLD COMPARISON")
    print(f"Period: {start_date} to {end_date}")
    print(f"{'='*80}\n")
    
    # FIX: Access strategy_metrics['overall'] but bh_metrics directly
    comparison_df = pd.DataFrame({
        'Strategy': {
            'Total Return': f"{strategy_metrics['overall']['total_return']*100:.2f}%",
            'Annualized Return': f"{strategy_metrics['overall']['annualized_return']*100:.2f}%",
            'Annualized Volatility': f"{strategy_metrics['overall']['annualized_volatility']*100:.2f}%",
            'Sharpe Ratio': f"{strategy_metrics['overall']['sharpe_ratio']:.2f}",
            'Sortino Ratio': f"{strategy_metrics['overall']['sortino_ratio']:.2f}",
            'Max Drawdown': f"{strategy_metrics['overall']['max_drawdown']*100:.2f}%",
            'Calmar Ratio': f"{strategy_metrics['overall']['calmar_ratio']:.2f}",
            'Final Value': f"${strategy_metrics['overall']['final_value']:,.0f}"
        },
        'Buy & Hold': { # <-- ALL 'overall' REMOVED FROM THIS BLOCK
            'Total Return': f"{bh_metrics['total_return']*100:.2f}%",
            'Annualized Return': f"{bh_metrics['annualized_return']*100:.2f}%",
            'Annualized Volatility': f"{bh_metrics['annualized_volatility']*100:.2f}%",
            'Sharpe Ratio': f"{bh_metrics['sharpe_ratio']:.2f}",
            'Sortino Ratio': f"{bh_metrics['sortino_ratio']:.2f}",
            'Max Drawdown': f"{bh_metrics['max_drawdown']*100:.2f}%",
            'Calmar Ratio': f"{bh_metrics['calmar_ratio']:.2f}",
            'Final Value': f"${bh_metrics['final_value']:,.0f}"
        }
    })
    
    # Add difference column
    # FIX: Access strategy_metrics['overall'] but bh_metrics directly
    comparison_df['Difference'] = ''
    for metric in comparison_df.index:
        if metric == 'Final Value':
            diff = strategy_metrics['overall']['final_value'] - bh_metrics['final_value'] # <-- FIXED
            comparison_df.loc[metric, 'Difference'] = f"${diff:,.0f}"
        elif 'Ratio' in metric:
            metric_key = metric.lower().replace(' ', '_')
            diff = strategy_metrics['overall'][metric_key] - bh_metrics[metric_key] # <-- FIXED
            comparison_df.loc[metric, 'Difference'] = f"{diff:+.2f}"
        else:
            metric_key = metric.lower().replace(' ', '_')
            diff = (strategy_metrics['overall'][metric_key] - bh_metrics[metric_key]) # <-- FIXED
            # Handle percentage metrics
            if 'return' in metric_key or 'volatility' in metric_key or 'drawdown' in metric_key:
                diff *= 100
            comparison_df.loc[metric, 'Difference'] = f"{diff:+.2f}{'%' if '%' in metric else ''}"
    
    print(comparison_df.to_string())
    
    # Determine winner
    # FIX: Access strategy_metrics['overall'] but bh_metrics directly
    print(f"\n{'='*80}")
    print("WINNER DETERMINATION")
    print(f"{'='*80}")
    
    strategy_wins = 0
    bh_wins = 0
    
    if strategy_metrics['overall']['total_return'] > bh_metrics['total_return']: # <-- FIXED
        print("✅ Strategy has HIGHER total return")
        strategy_wins += 1
    else:
        print("❌ Buy & Hold has HIGHER total return")
        bh_wins += 1
    
    if strategy_metrics['overall']['sharpe_ratio'] > bh_metrics['sharpe_ratio']: # <-- FIXED
        print("✅ Strategy has BETTER risk-adjusted returns (Sharpe)")
        strategy_wins += 1
    else:
        print("❌ Buy & Hold has BETTER risk-adjusted returns (Sharpe)")
        bh_wins += 1
    
    if strategy_metrics['overall']['max_drawdown'] > bh_metrics['max_drawdown']: # <-- FIXED
        print("✅ Strategy has LOWER maximum drawdown")
        strategy_wins += 1
    else:
        print("❌ Buy & Hold has LOWER maximum drawdown")
        bh_wins += 1
    
    if strategy_metrics['overall']['annualized_volatility'] < bh_metrics['annualized_volatility']: # <-- FIXED
        print("✅ Strategy has LOWER volatility")
        strategy_wins += 1
    else:
        print("❌ Buy & Hold has LOWER volatility")
        bh_wins += 1
    
    print(f"\n{'='*80}")
    if strategy_wins > bh_wins:
        print(f"🏆 STRATEGY WINS: {strategy_wins} vs {bh_wins}")
        print("✅ Your event-driven strategy OUTPERFORMS buy-and-hold!")
    elif bh_wins > bh_wins:
        print(f"🏆 BUY & HOLD WINS: {bh_wins} vs {strategy_wins}")
        print("❌ Buy-and-hold strategy OUTPERFORMS your strategy")
    else:
        print(f"🤝 TIE: {strategy_wins} vs {bh_wins}")
        print("⚖️  Both strategies perform equally")
    print(f"{'='*80}\n")


CALCULATING BUY-AND-HOLD BENCHMARK


$NVDA: possibly delisted; no price data found  (1d 2010-01-01 -> 2019-12-31) (Yahoo error = "Request Failed")
$MSFT: possibly delisted; no price data found  (1d 2010-01-01 -> 2019-12-31) (Yahoo error = "Error occurred while retrieving timeseries, keys: [RedisKey [key=TF[1,YF_HIST][5970], cluster=finance]]")
$AVGO: possibly delisted; no price data found  (1d 2010-01-01 -> 2019-12-31) (Yahoo error = "Error occurred while retrieving timeseries, keys: [RedisKey [key=TF[1,YF_HIST,DV][89843], cluster=finance]]")



✅ Buy-and-Hold calculated for 7 stocks
Final Value: $4,244,084.57
Total Return: 748.82%


STRATEGY VS BUY-AND-HOLD COMPARISON
Period: 2010-01-01 to 2019-12-31

                       Strategy  Buy & Hold   Difference
Total Return             70.99%     748.82%      -677.83
Annualized Return         5.54%      23.90%       -18.36
Annualized Volatility     7.79%      24.14%       -16.36
Sharpe Ratio               0.71        0.99        -0.28
Sortino Ratio              0.56        1.47        -0.92
Max Drawdown            -17.32%     -24.68%        +7.36
Calmar Ratio               0.32        0.97        -0.65
Final Value            $854,931  $4,244,085  $-3,389,153

WINNER DETERMINATION
❌ Buy & Hold has HIGHER total return
❌ Buy & Hold has BETTER risk-adjusted returns (Sharpe)
✅ Strategy has LOWER maximum drawdown
✅ Strategy has LOWER volatility

🤝 TIE: 2 vs 2
⚖️  Both strategies perform equally



In [171]:
# ============================================================================
# RUN STRATEGY VS BUY-AND-HOLD COMPARISON
# ============================================================================

# Assuming you have already run your strategy backtest and stored results in 'results'
# If not, run your backtest first:
# results = run_backtest(start_date=start_date, end_date=end_date, stock_list=stock_list, **other_variables)

# Calculate Buy-and-Hold benchmark
bh_results = calculate_buy_and_hold(
    stock_list=stock_list,
    start_date=start_date,
    end_date=end_date,
    initial_capital=other_variables['initial_capital']
)

# Compare Strategy vs Buy-and-Hold
compare_strategy_vs_benchmark(
    strategy_metrics=results['metrics'],
    bh_metrics=bh_results,
    start_date=start_date,
    end_date=end_date
)

In [172]:
#add monte carlo
# ============================================================================
# MONTE CARLO SIMULATION: PARAMETER UNCERTAINTY
# ============================================================================

def monte_carlo_parameter_test(stock_data: Dict[str, pd.DataFrame], vix_data: pd.DataFrame,
                               base_config: StrategyConfig,
                               n_simulations: int = 500) -> pd.DataFrame:
    """
    Test strategy with random parameter variations
    
    Randomly samples parameters from reasonable ranges to test robustness
    """
    print(f"\n{'='*60}")
    print(f"MONTE CARLO: PARAMETER UNCERTAINTY TEST ({n_simulations} runs)")
    print(f"{'='*60}\n")
    
    # Define parameter ranges
    param_ranges = {
        'ema_fast': list(range(10, 31, 5)),           # [10, 15, 20, 25, 30]
        'ema_slow': list(range(30, 71, 10)),          # [30, 40, 50, 60, 70]
        # 'rsi_period': list(range(10, 21, 2)),         # [10, 12, 14, 16, 18, 20]
        # 'adx_threshold': np.arange(15, 36, 5),        # [15, 20, 25, 30, 35]
        # 'rsi_long_threshold': np.arange(45, 61, 5),   # [45, 50, 55, 60]
        'stop_loss_pct': np.arange(0.015, 0.031, 0.005),     # [1.5%, 2%, 2.5%, 3%]
        'trailing_stop_pct': np.arange(0.01, 0.026, 0.005),  # [1%, 1.5%, 2%, 2.5%]
        'max_positions': list(range(5, 16, 5))        # [5, 10, 15]
    }
    
    results = []
    failed_count = 0
    
    for i in tqdm(range(n_simulations), desc="Running parameter variations"):
        # Random parameter selection
        test_config = StrategyConfig(
            ema_fast=np.random.choice(param_ranges['ema_fast']),
            ema_slow=np.random.choice(param_ranges['ema_slow']),
            # rsi_period=np.random.choice(param_ranges['rsi_period']),
            # adx_threshold=np.random.choice(param_ranges['adx_threshold']),
            # rsi_long_threshold=np.random.choice(param_ranges['rsi_long_threshold']),
            stop_loss_pct=np.random.choice(param_ranges['stop_loss_pct']),
            trailing_stop_pct=np.random.choice(param_ranges['trailing_stop_pct']),
            max_positions=np.random.choice(param_ranges['max_positions']),
            initial_capital=base_config.initial_capital,
            transaction_cost=base_config.transaction_cost,
            leverage=base_config.leverage,
            train_start=base_config.train_start,
            train_end=base_config.train_end
        )
        
        # Ensure fast < slow
        if test_config.ema_fast >= test_config.ema_slow:
            failed_count += 1
            continue
        
        try:
            # Run backtest with new parameters
            backtester = PortfolioBacktester(test_config, stock_data, vix_data)
            backtester.run()
            
            metrics = calculate_portfolio_metrics(backtester)
            # --- FIX: Access metrics from the 'overall' sub-dictionary ---
            if 'overall' not in metrics:
                failed_count += 1
                continue
            n = metrics['overall']
            
            results.append({
                'run': i,
                'ema_fast': test_config.ema_fast,
                'ema_slow': test_config.ema_slow,
                # 'adx_threshold': test_config.adx_threshold,
                'stop_loss_pct': test_config.stop_loss_pct,
                'max_positions': test_config.max_positions,
                'total_return': n['total_return'],
                'annualized_return': n['annualized_return'],
                'sharpe_ratio': n['sharpe_ratio'],
                'sortino_ratio': n['sortino_ratio'],
                'max_drawdown': n['max_drawdown'],
                'calmar_ratio': n['calmar_ratio'],
                'win_rate': n['win_rate'],
                'total_trades': n['total_trades']
            })
            
        except Exception as e:
            failed_count += 1
            continue
    
    results_df = pd.DataFrame(results)
    
    print(f"\n✅ Completed: {len(results_df)} successful, {failed_count} failed")
    print(f"\nParameter Uncertainty Results:")
    print(f"  Mean Sharpe Ratio: {results_df['sharpe_ratio'].mean():.2f}")
    print(f"  Median Sharpe Ratio: {results_df['sharpe_ratio'].median():.2f}")
    print(f"  Std Dev Sharpe Ratio: {results_df['sharpe_ratio'].std():.2f}")
    print(f"  % Profitable Runs: {(results_df['total_return'] > 0).sum() / len(results_df) * 100:.1f}%")
    print(f"  % Sharpe > 1.0: {(results_df['sharpe_ratio'] > 1.0).sum() / len(results_df) * 100:.1f}%")
    
    return results_df

print("✅ Monte Carlo parameter test function defined!")

✅ Monte Carlo parameter test function defined!


In [173]:
# ============================================================================
# MONTE CARLO SIMULATION: BOOTSTRAP RESAMPLING
# ============================================================================

def monte_carlo_bootstrap_test(strategy_results: pd.DataFrame,
                               n_samples: int = 1000) -> pd.DataFrame:
    """
    Bootstrap resampling of daily returns to test statistical significance
    
    Resamples with replacement to generate confidence intervals
    """
    print(f"\n{'='*60}")
    print(f"MONTE CARLO: BOOTSTRAP RESAMPLING ({n_samples} samples)")
    print(f"{'='*60}\n")
    
    # Get strategy returns
    daily_returns = strategy_results['daily_return'].dropna()
    n_days = len(daily_returns)
    
    if n_days < 10:
        print("⚠️ Insufficient data for bootstrap")
        return pd.DataFrame()
    
    bootstrap_results = []
    
    for i in tqdm(range(n_samples), desc="Bootstrap resampling"):
        # Resample with replacement
        resampled_returns = np.random.choice(daily_returns, size=n_days, replace=True)
        
        # Calculate metrics
        total_return = np.exp(resampled_returns.sum()) - 1
        annual_return = (1 + total_return) ** (252 / n_days) - 1
        annual_vol = resampled_returns.std() * np.sqrt(252)
        sharpe = annual_return / annual_vol if annual_vol > 0 else 0
        
        # Downside deviation
        downside = resampled_returns[resampled_returns < 0]
        downside_std = downside.std() * np.sqrt(252) if len(downside) > 0 else 0.0001
        sortino = annual_return / downside_std if downside_std > 0 else 0
        
        bootstrap_results.append({
            'sample': i,
            'total_return': total_return,
            'annualized_return': annual_return,
            'annualized_volatility': annual_vol,
            'sharpe_ratio': sharpe,
            'sortino_ratio': sortino
        })
    
    bootstrap_df = pd.DataFrame(bootstrap_results)
    
    print(f"\n✅ Bootstrap Resampling Results:")
    print(f"  Mean Return: {bootstrap_df['annualized_return'].mean()*100:.2f}%")
    print(f"  95% CI Return: [{bootstrap_df['annualized_return'].quantile(0.025)*100:.2f}%, "
          f"{bootstrap_df['annualized_return'].quantile(0.975)*100:.2f}%]")
    print(f"  Mean Sharpe: {bootstrap_df['sharpe_ratio'].mean():.2f}")
    print(f"  95% CI Sharpe: [{bootstrap_df['sharpe_ratio'].quantile(0.025):.2f}, "
          f"{bootstrap_df['sharpe_ratio'].quantile(0.975):.2f}]")
    print(f"  Probability of Positive Return: {(bootstrap_df['annualized_return'] > 0).sum() / len(bootstrap_df) * 100:.1f}%")
    
    return bootstrap_df

print("✅ Monte Carlo bootstrap function defined!")

✅ Monte Carlo bootstrap function defined!


In [174]:
# ============================================================================
# MONTE CARLO SIMULATION: SYNTHETIC PRICE PATHS
# ============================================================================

def monte_carlo_synthetic_paths(stock_data: Dict[str, pd.DataFrame], # <-- ADDED
                                vix_data: pd.DataFrame,           # <-- ADDED
                                base_config: StrategyConfig,
                                n_paths: int = 100) -> pd.DataFrame:
    """
    Test strategy on synthetic price paths using Geometric Brownian Motion
    
    Generates random price paths with same statistical properties as real data
    """
    print(f"\n{'='*60}")
    print(f"MONTE CARLO: SYNTHETIC PRICE PATHS ({n_paths} paths)")
    print(f"{'='*60}\n")
    
    # Get trading dates and number of days from VIX data
    trading_dates = vix_data.index
    n_days = len(trading_dates)

    print("Estimating parameters from existing data...")
    stock_params = {}
    for symbol, df in stock_data.items():
        # Ensure df index matches vix index
        df = df.reindex(trading_dates).fillna(method='ffill')
        if df.isnull().any().any():
            continue # Skip if still has NaNs
            
        returns = np.log(df['Close'] / df['Close'].shift(1)).dropna()
        if returns.empty:
            continue
            
        stock_params[symbol] = {
            'mu': returns.mean(),
            'sigma': returns.std(),
            'initial_price': df['Close'].iloc[0],
        }
    
    if len(stock_params) == 0:
        print("⚠️ No valid stock data to estimate parameters")
        return pd.DataFrame()

    print(f"Generating paths for {len(stock_params)} stocks...")
    
    # Run simulations
    synthetic_results = []
    failed_count = 0
    
    for path_num in tqdm(range(n_paths), desc="Generating synthetic paths"):
        # Generate synthetic data for each stock
        synthetic_stock_data = {}
        
        for symbol, params in stock_params.items():
            # Geometric Brownian Motion
            dt = 1/252 # Assuming 252 trading days
            price_path = [params['initial_price']]
            
            for _ in range(n_days - 1):
                drift = (params['mu'] - 0.5 * params['sigma']**2) * dt
                diffusion = params['sigma'] * np.sqrt(dt) * np.random.normal()
                price = price_path[-1] * np.exp(drift + diffusion)
                price_path.append(price)
            
            # Create synthetic dataframe
            synthetic_df = pd.DataFrame({
                'Close': price_path,
                # Add High/Low/Volume for indicators
                'High': [p * (1 + abs(np.random.normal(0, 0.01))) for p in price_path],
                'Low': [p * (1 - abs(np.random.normal(0, 0.01))) for p in price_path],
                'Volume': [np.random.randint(1e6, 1e7) for _ in price_path]
            }, index=trading_dates) # <-- Use VIX index
            
            # Calculate indicators
            try:
                df_with_indicators = calculate_indicators(synthetic_df, base_config)
                synthetic_stock_data[symbol] = df_with_indicators
            except:
                continue
        
        if len(synthetic_stock_data) < 1:
            failed_count += 1
            continue
        
        # Run backtest on synthetic data
        try:
            # --- FIX: Pass 'vix_data' to the backtester ---
            backtester = PortfolioBacktester(base_config, synthetic_stock_data, vix_data) # <-- ADDED
            backtester.run()
            
            metrics = calculate_portfolio_metrics(backtester)
            
            # --- FIX: Access metrics from the 'overall' sub-dictionary ---
            if 'overall' not in metrics:
                failed_count += 1
                continue
            
            m = metrics['overall'] # Helper variable
            
            synthetic_results.append({
                'path': path_num,
                'total_return': m['total_return'],           # <-- FIXED
                'annualized_return': m['annualized_return'], # <-- FIXED
                'sharpe_ratio': m['sharpe_ratio'],           # <-- FIXED
                'sortino_ratio': m['sortino_ratio'],         # <-- FIXED
                'max_drawdown': m['max_drawdown'],         # <-- FIXED
                'calmar_ratio': m['calmar_ratio'],           # <-- FIXED
                'total_trades': m['total_trades']          # <-- FIXED
            })
            
        except Exception as e:
            failed_count += 1
            continue
    
    synthetic_df = pd.DataFrame(synthetic_results)

    
    if len(synthetic_df) > 0:
        print(f"\n✅ Completed: {len(synthetic_df)} successful, {failed_count} failed")
        print(f"\nSynthetic Paths Results:")
        print(f"  Mean Sharpe Ratio: {synthetic_df['sharpe_ratio'].mean():.2f}")
        print(f"  % Profitable Runs: {(synthetic_df['total_return'] > 0).sum() / len(synthetic_df) * 100:.1f}%")
        print(f"  % Sharpe > 1.0: {(synthetic_df['sharpe_ratio'] > 1.0).sum() / len(synthetic_df) * 100:.1f}%")
    else:
        print("⚠️ All synthetic simulations failed")
    
    return synthetic_df

print("✅ Monte Carlo synthetic paths function defined!")

✅ Monte Carlo synthetic paths function defined!


In [175]:
# ============================================================================
# MONTE CARLO VISUALIZATION
# ============================================================================

def plot_monte_carlo_results(param_results: pd.DataFrame,
                             bootstrap_results: pd.DataFrame,
                             synthetic_results: pd.DataFrame,
                             actual_sharpe: float):
    """
    Visualize all Monte Carlo simulation results
    """
    fig, axes = plt.subplots(2, 3, figsize=(20, 12))
    
    # 1. Parameter Test - Sharpe Distribution
    if len(param_results) > 0:
        axes[0, 0].hist(param_results['sharpe_ratio'], bins=30, color='blue', alpha=0.7, edgecolor='black')
        axes[0, 0].axvline(param_results['sharpe_ratio'].mean(), color='red', linestyle='--', 
                          linewidth=2, label=f'Mean: {param_results["sharpe_ratio"].mean():.2f}')
        axes[0, 0].axvline(actual_sharpe, color='green', linestyle='--', 
                          linewidth=2, label=f'Actual: {actual_sharpe:.2f}')
        axes[0, 0].axvline(1.0, color='gray', linestyle=':', linewidth=1, label='Target: 1.0')
        axes[0, 0].set_title('Parameter Uncertainty: Sharpe Ratio', fontsize=12, fontweight='bold')
        axes[0, 0].set_xlabel('Sharpe Ratio')
        axes[0, 0].set_ylabel('Frequency')
        axes[0, 0].legend()
        axes[0, 0].grid(True, alpha=0.3)
    
    # 2. Parameter Test - Returns Distribution
    if len(param_results) > 0:
        axes[0, 1].hist(param_results['total_return'] * 100, bins=30, color='green', alpha=0.7, edgecolor='black')
        axes[0, 1].axvline(param_results['total_return'].mean() * 100, color='red', linestyle='--', 
                          linewidth=2, label=f'Mean: {param_results["total_return"].mean()*100:.1f}%')
        axes[0, 1].axvline(0, color='black', linestyle='-', linewidth=1)
        axes[0, 1].set_title('Parameter Uncertainty: Total Return', fontsize=12, fontweight='bold')
        axes[0, 1].set_xlabel('Total Return (%)')
        axes[0, 1].set_ylabel('Frequency')
        axes[0, 1].legend()
        axes[0, 1].grid(True, alpha=0.3)
    
    # 3. Parameter Test - Max Drawdown
    if len(param_results) > 0:
        axes[0, 2].hist(param_results['max_drawdown'] * 100, bins=30, color='red', alpha=0.7, edgecolor='black')
        axes[0, 2].axvline(param_results['max_drawdown'].mean() * 100, color='blue', linestyle='--', 
                          linewidth=2, label=f'Mean: {param_results["max_drawdown"].mean()*100:.1f}%')
        axes[0, 2].axvline(-15, color='darkred', linestyle=':', linewidth=2, label='Kill Switch: -15%')
        axes[0, 2].set_title('Parameter Uncertainty: Max Drawdown', fontsize=12, fontweight='bold')
        axes[0, 2].set_xlabel('Max Drawdown (%)')
        axes[0, 2].set_ylabel('Frequency')
        axes[0, 2].legend()
        axes[0, 2].grid(True, alpha=0.3)
    
    # 4. Bootstrap - Sharpe Distribution
    if len(bootstrap_results) > 0:
        axes[1, 0].hist(bootstrap_results['sharpe_ratio'], bins=30, color='purple', alpha=0.7, edgecolor='black')
        axes[1, 0].axvline(bootstrap_results['sharpe_ratio'].mean(), color='red', linestyle='--', 
                          linewidth=2, label=f'Mean: {bootstrap_results["sharpe_ratio"].mean():.2f}')
        ci_lower = bootstrap_results['sharpe_ratio'].quantile(0.025)
        ci_upper = bootstrap_results['sharpe_ratio'].quantile(0.975)
        axes[1, 0].axvline(ci_lower, color='orange', linestyle=':', linewidth=1, label=f'95% CI')
        axes[1, 0].axvline(ci_upper, color='orange', linestyle=':', linewidth=1)
        axes[1, 0].set_title('Bootstrap: Sharpe Ratio Distribution', fontsize=12, fontweight='bold')
        axes[1, 0].set_xlabel('Sharpe Ratio')
        axes[1, 0].set_ylabel('Frequency')
        axes[1, 0].legend()
        axes[1, 0].grid(True, alpha=0.3)
    
    # 5. Bootstrap - Return Distribution
    if len(bootstrap_results) > 0:
        axes[1, 1].hist(bootstrap_results['annualized_return'] * 100, bins=30, color='cyan', alpha=0.7, edgecolor='black')
        axes[1, 1].axvline(bootstrap_results['annualized_return'].mean() * 100, color='red', linestyle='--', 
                          linewidth=2, label=f'Mean: {bootstrap_results["annualized_return"].mean()*100:.1f}%')
        axes[1, 1].axvline(0, color='black', linestyle='-', linewidth=1)
        axes[1, 1].set_title('Bootstrap: Annualized Return', fontsize=12, fontweight='bold')
        axes[1, 1].set_xlabel('Annualized Return (%)')
        axes[1, 1].set_ylabel('Frequency')
        axes[1, 1].legend()
        axes[1, 1].grid(True, alpha=0.3)
    
    # 6. Synthetic Paths - Sharpe Distribution
    if len(synthetic_results) > 0:
        axes[1, 2].hist(synthetic_results['sharpe_ratio'], bins=30, color='orange', alpha=0.7, edgecolor='black')
        axes[1, 2].axvline(synthetic_results['sharpe_ratio'].mean(), color='red', linestyle='--', 
                          linewidth=2, label=f'Mean: {synthetic_results["sharpe_ratio"].mean():.2f}')
        axes[1, 2].axvline(actual_sharpe, color='green', linestyle='--', 
                          linewidth=2, label=f'Actual: {actual_sharpe:.2f}')
        axes[1, 2].set_title('Synthetic Paths: Sharpe Ratio', fontsize=12, fontweight='bold')
        axes[1, 2].set_xlabel('Sharpe Ratio')
        axes[1, 2].set_ylabel('Frequency')
        axes[1, 2].legend()
        axes[1, 2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Summary statistics
    print(f"\n{'='*80}")
    print("MONTE CARLO SUMMARY")
    print(f"{'='*80}\n")
    
    if len(param_results) > 0:
        print("Parameter Uncertainty Test:")
        print(f"  Simulations: {len(param_results)}")
        print(f"  Mean Sharpe: {param_results['sharpe_ratio'].mean():.2f} ± {param_results['sharpe_ratio'].std():.2f}")
        print(f"  Sharpe Range: [{param_results['sharpe_ratio'].min():.2f}, {param_results['sharpe_ratio'].max():.2f}]")
        print(f"  % Sharpe > Actual ({actual_sharpe:.2f}): {(param_results['sharpe_ratio'] > actual_sharpe).sum() / len(param_results) * 100:.1f}%")
        print()
    
    if len(bootstrap_results) > 0:
        print("Bootstrap Resampling:")
        print(f"  Samples: {len(bootstrap_results)}")
        print(f"  Mean Return: {bootstrap_results['annualized_return'].mean()*100:.2f}%")
        print(f"  95% CI: [{bootstrap_results['annualized_return'].quantile(0.025)*100:.2f}%, {bootstrap_results['annualized_return'].quantile(0.975)*100:.2f}%]")
        print(f"  P(Return > 0): {(bootstrap_results['annualized_return'] > 0).sum() / len(bootstrap_results) * 100:.1f}%")
        print()
    
    if len(synthetic_results) > 0:
        print("Synthetic Price Paths:")
        print(f"  Paths: {len(synthetic_results)}")
        print(f"  Mean Sharpe: {synthetic_results['sharpe_ratio'].mean():.2f}")
        print(f"  % Profitable: {(synthetic_results['total_return'] > 0).sum() / len(synthetic_results) * 100:.1f}%")
        print(f"  Strategy works on random data: {'✅ YES' if synthetic_results['sharpe_ratio'].mean() > 0 else '❌ NO'}")
    
    print(f"\n{'='*80}\n")

print("✅ Monte Carlo visualization function defined!")

✅ Monte Carlo visualization function defined!


✅ All variables extracted successfully
   Config: <class '__main__.StrategyConfig'>
   Stock data: 7 stocks
   VIX data: 2506 days
   Strategy Sharpe: 0.71

STARTING QUICK MONTE CARLO ANALYSIS (10 simulations each)

Test 1/3: Parameter Uncertainty (10 simulations)...

MONTE CARLO: PARAMETER UNCERTAINTY TEST (10 runs)



Running parameter variations:   0%|          | 0/10 [00:00<?, ?it/s]


SIMPLIFIED EVENT-DRIVEN PORTFOLIO BACKTEST
Period: 2010-01-01 to 2019-12-31
Universe: 7 stocks
Initial Capital: $500,000



Processing days:   0%|          | 0/2506 [00:00<?, ?it/s]


BACKTEST COMPLETE
Total Events: 4182
Total Trades: 195
Final Portfolio Value: $936,961.50
Total Return: 87.39%


SIMPLIFIED EVENT-DRIVEN PORTFOLIO BACKTEST
Period: 2010-01-01 to 2019-12-31
Universe: 7 stocks
Initial Capital: $500,000



Processing days:   0%|          | 0/2506 [00:00<?, ?it/s]


BACKTEST COMPLETE
Total Events: 4180
Total Trades: 193
Final Portfolio Value: $872,898.85
Total Return: 74.58%


SIMPLIFIED EVENT-DRIVEN PORTFOLIO BACKTEST
Period: 2010-01-01 to 2019-12-31
Universe: 7 stocks
Initial Capital: $500,000



Processing days:   0%|          | 0/2506 [00:00<?, ?it/s]


BACKTEST COMPLETE
Total Events: 4185
Total Trades: 198
Final Portfolio Value: $797,093.95
Total Return: 59.42%


SIMPLIFIED EVENT-DRIVEN PORTFOLIO BACKTEST
Period: 2010-01-01 to 2019-12-31
Universe: 7 stocks
Initial Capital: $500,000



Processing days:   0%|          | 0/2506 [00:00<?, ?it/s]


BACKTEST COMPLETE
Total Events: 4180
Total Trades: 193
Final Portfolio Value: $872,898.85
Total Return: 74.58%


SIMPLIFIED EVENT-DRIVEN PORTFOLIO BACKTEST
Period: 2010-01-01 to 2019-12-31
Universe: 7 stocks
Initial Capital: $500,000



Processing days:   0%|          | 0/2506 [00:00<?, ?it/s]


BACKTEST COMPLETE
Total Events: 4180
Total Trades: 193
Final Portfolio Value: $872,898.85
Total Return: 74.58%


SIMPLIFIED EVENT-DRIVEN PORTFOLIO BACKTEST
Period: 2010-01-01 to 2019-12-31
Universe: 7 stocks
Initial Capital: $500,000



Processing days:   0%|          | 0/2506 [00:00<?, ?it/s]


BACKTEST COMPLETE
Total Events: 4180
Total Trades: 193
Final Portfolio Value: $872,898.85
Total Return: 74.58%


SIMPLIFIED EVENT-DRIVEN PORTFOLIO BACKTEST
Period: 2010-01-01 to 2019-12-31
Universe: 7 stocks
Initial Capital: $500,000



Processing days:   0%|          | 0/2506 [00:00<?, ?it/s]


BACKTEST COMPLETE
Total Events: 4185
Total Trades: 198
Final Portfolio Value: $797,093.95
Total Return: 59.42%


SIMPLIFIED EVENT-DRIVEN PORTFOLIO BACKTEST
Period: 2010-01-01 to 2019-12-31
Universe: 7 stocks
Initial Capital: $500,000



Processing days:   0%|          | 0/2506 [00:00<?, ?it/s]


BACKTEST COMPLETE
Total Events: 4182
Total Trades: 195
Final Portfolio Value: $936,961.50
Total Return: 87.39%


SIMPLIFIED EVENT-DRIVEN PORTFOLIO BACKTEST
Period: 2010-01-01 to 2019-12-31
Universe: 7 stocks
Initial Capital: $500,000



Processing days:   0%|          | 0/2506 [00:00<?, ?it/s]


BACKTEST COMPLETE
Total Events: 4185
Total Trades: 198
Final Portfolio Value: $797,093.95
Total Return: 59.42%


✅ Completed: 9 successful, 1 failed

Parameter Uncertainty Results:
  Mean Sharpe Ratio: 0.48
  Median Sharpe Ratio: 0.48
  Std Dev Sharpe Ratio: 0.05
  % Profitable Runs: 100.0%
  % Sharpe > 1.0: 0.0%
✅ Parameter test complete: 9 successful runs

Test 2/3: Bootstrap Resampling (10 samples)...

MONTE CARLO: BOOTSTRAP RESAMPLING (10 samples)



Bootstrap resampling:   0%|          | 0/10 [00:00<?, ?it/s]


✅ Bootstrap Resampling Results:
  Mean Return: 6.56%
  95% CI Return: [3.47%, 9.46%]
  Mean Sharpe: 0.83
  95% CI Sharpe: [0.44, 1.21]
  Probability of Positive Return: 100.0%
✅ Bootstrap test complete: 10 samples

Test 3/3: Synthetic Price Paths (10 paths)...

MONTE CARLO: SYNTHETIC PRICE PATHS (10 paths)

Estimating parameters from existing data...
⚠️ No valid stock data to estimate parameters
✅ Synthetic test complete: 0 paths

Generating visualizations...


MONTE CARLO SUMMARY

Parameter Uncertainty Test:
  Simulations: 9
  Mean Sharpe: 0.48 ± 0.05
  Sharpe Range: [0.42, 0.55]
  % Sharpe > Actual (0.71): 0.0%

Bootstrap Resampling:
  Samples: 10
  Mean Return: 6.56%
  95% CI: [3.47%, 9.46%]
  P(Return > 0): 100.0%



✅ Visualizations complete!

QUICK MONTE CARLO RESULTS

Parameter Test (n=9):
  Mean Sharpe: 0.48
  Range: [0.42, 0.55]
  % Profitable: 100%

Bootstrap Test (n=10):
  Mean Return: 6.6%
  Sharpe Range: [0.37, 1.25]




In [176]:
# ============================================================================
# QUICK MONTE CARLO ANALYSIS (10 SIMULATIONS FOR TESTING)
# ============================================================================

# Step 1: Verify results exist
if 'results' not in locals() and 'results' not in globals():
    raise ValueError("❌ Please run the backtest first: results = run_backtest(...)")

# Step 2: Extract required variables
try:
    config = results['config']
    backtester = results['backtester']
    stock_data = backtester.stock_data
    vix_data = backtester.vix_data # <--- ADD THIS
    strategy_metrics = results['metrics']
    
    print("✅ All variables extracted successfully")
    print(f"   Config: {type(config)}")
    print(f"   Stock data: {len(stock_data)} stocks")
    print(f"   VIX data: {len(vix_data)} days") # <--- ADD THIS
    # --- FIX: Access 'overall' dictionary ---
    print(f"   Strategy Sharpe: {strategy_metrics['overall']['sharpe_ratio']:.2f}")
    
except KeyError as e:
    raise ValueError(f"❌ Missing key in results: {e}. Did you fix the 'overall' nesting?")

# Step 3: Run Monte Carlo Tests with SMALL sample sizes
print(f"\n{'='*60}")
print("STARTING QUICK MONTE CARLO ANALYSIS (10 simulations each)")
print(f"{'='*60}\n")

# Test 1: Parameter Uncertainty (10 runs)
try:
    print("Test 1/3: Parameter Uncertainty (10 simulations)...")
    param_results = monte_carlo_parameter_test(
        stock_data=stock_data,
        vix_data=vix_data, # <--- ADD THIS
        base_config=config,
        n_simulations=10
    )
    print(f"✅ Parameter test complete: {len(param_results)} successful runs")
except Exception as e:
    print(f"❌ Parameter test failed: {e}")
    param_results = pd.DataFrame()

# Test 2: Bootstrap Resampling (10 samples)
# (This function is correct, no changes needed)
try:
    print("\nTest 2/3: Bootstrap Resampling (10 samples)...")
    bootstrap_results = monte_carlo_bootstrap_test(
        strategy_results=strategy_metrics['daily_data'],
        n_samples=10 
    )
    print(f"✅ Bootstrap test complete: {len(bootstrap_results)} samples")
except Exception as e:
    print(f"❌ Bootstrap test failed: {e}")
    bootstrap_results = pd.DataFrame()

# Test 3: Synthetic Paths (10 paths)
try:
    print("\nTest 3/3: Synthetic Price Paths (10 paths)...")
    # --- FIX: Updated function signature ---
    synthetic_results = monte_carlo_synthetic_paths(
        stock_data=stock_data, # <--- ADD THIS
        vix_data=vix_data,   # <--- ADD THIS
        base_config=config,
        n_paths=10
    )
    print(f"✅ Synthetic test complete: {len(synthetic_results)} paths")
except Exception as e:
    print(f"❌ Synthetic test failed: {e}")
    synthetic_results = pd.DataFrame()

# Step 4: Visualize Results
if len(param_results) > 0 or len(bootstrap_results) > 0 or len(synthetic_results) > 0:
    print("\nGenerating visualizations...")
    plot_monte_carlo_results(
        param_results=param_results,
        bootstrap_results=bootstrap_results,
        synthetic_results=synthetic_results,
        # --- FIX: Access 'overall' dictionary ---
        actual_sharpe=strategy_metrics['overall']['sharpe_ratio'] 
    )
    print("✅ Visualizations complete!")
else:
    print("⚠️ No results to visualize")

# Print quick summary
print(f"\n{'='*60}")
print("QUICK MONTE CARLO RESULTS")
print(f"{'='*60}")

if len(param_results) > 0:
    print(f"\nParameter Test (n={len(param_results)}):")
    print(f"  Mean Sharpe: {param_results['sharpe_ratio'].mean():.2f}")
    print(f"  Range: [{param_results['sharpe_ratio'].min():.2f}, {param_results['sharpe_ratio'].max():.2f}]")
    print(f"  % Profitable: {(param_results['total_return'] > 0).sum() / len(param_results) * 100:.0f}%")

if len(bootstrap_results) > 0:
    print(f"\nBootstrap Test (n={len(bootstrap_results)}):")
    print(f"  Mean Return: {bootstrap_results['annualized_return'].mean()*100:.1f}%")
    print(f"  Sharpe Range: [{bootstrap_results['sharpe_ratio'].min():.2f}, {bootstrap_results['sharpe_ratio'].max():.2f}]")

if len(synthetic_results) > 0:
    print(f"\nSynthetic Test (n={len(synthetic_results)}):")
    print(f"  Mean Sharpe: {synthetic_results['sharpe_ratio'].mean():.2f}")
    print(f"  % Profitable: {(synthetic_results['total_return'] > 0).sum() / len(synthetic_results) * 100:.0f}%")

print(f"\n{'='*60}\n")